# 导入模块

In [265]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import gc
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [266]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 导入订单数据

In [267]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag, tojo.app_type, tprm.rent_received 已收租金 
, tolog.status 物流状态
,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
-- , ymos.money, ymos.part_payment, ymos.refund_date, ymos.reality_refund_date, ymos.sort, ymos.status 分期租金支付状态
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 租后分期表
-- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_credit.risk risk on risk.order_id=om.id


where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )               -- 近20天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-07-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-31'
-- and hour(om.create_time) >= 11
-- and hour(om.create_time) < 17
'''

df_order = clean.query(sql1)
df_order.shape

(322336, 52)

In [268]:
sql_modelname = '''
-- 商品表
select
tp.id, tp.classify_id, tp.id 商品ID, tpmn.name phone_name
from db_digua_business.t_product tp
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
'''
df_modelname = clean.query(sql_modelname)
df_order = df_order.merge(df_modelname, left_on='product_id', right_on='id',how='left')

In [269]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
-- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected

from db_credit.risk
'''
df_risk = clean.query(sql_risk)
# df_risk.loc[:, 'time_r'] = pd.to_datetime(df_risk.time.dt.date,errors="coerce")
# df_risk = df_risk.sort_values(by='time', ascending=False).groupby('id_card_r').head(1)

In [270]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = clean.query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
# df_re = df_re.sort_values(by='time', ascending=False).groupby('id_card_re').head(1)

In [271]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = clean.query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [272]:
sql_name = '''
SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
FROM db_digua_business.t_user_verify_order_record tuvor
left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
where tuvor.del_flag = 0 ORDER BY tuvor.update_time
'''
df_name = clean.query(sql_name)

In [273]:
# 首逾
sql_stages = '''
select 
tos.order_id, tos.sort, tos.refund_date, tos.reality_refund_date
from db_digua_business.t_order_stages tos
where tos.sort = 2
'''
df_stages = clean.query(sql_stages)

In [274]:
sql_wl = '''
select
date_format(om.create_time, '%Y-%m-%d') 下单日期, tort.order_num order_number, tort.task_status
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(4006, 3)

In [275]:
# sql_ywl = '''
# select

# from db_digua_business.t_order om
# left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
# '''

# 提取订单风控数据

# 合并数据

In [276]:
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(322333)

# 处理日期

In [277]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
df['hour'] = df['create_time'].dt.hour
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')

# 渠道补充

In [278]:
# qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
# qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)
# qudao_name_df.rename(columns={'订单号': '订单号1'}, inplace=True)
# df =  df.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号1',how = 'left')
# df["channel_name"] = np.where(df.渠道名称.notnull(),df.渠道名称,df["channel_name"])

# 处理备注信息

In [279]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip().str.split('"').str[0]
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")
df.loc[:,"颜色"]=df.apply(lambda x:clean.getcolor(x["sku_attributes"]),axis=1)
df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:, '商品ID'] = df.商品ID.astype(str)+'_'+df.phone_name
# 物流状态=5，退货拒签
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['phone_name'] + df['内存']

# 渠道归属

In [280]:

df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)


In [416]:
# df[df['来源渠道'].isin(['搜索下挂-商品', '搜索-vivo'])][['order_number','来源渠道', '归属渠道', 'channel_type_id', 'order_type']].drop_duplicates()
df[(df.channel_type_id.isin ([48, 81])) & (df.order_type=='ZFB_ORDER') & (df['来源渠道'] != '灯火联投测试')].来源渠道.value_counts()
# df.归属渠道.value_counts()

来源渠道
顶部搜索框的搜索结果页                  12809
搜索下挂-商品                       6493
搜索-租手机                        1794
搜索-租苹果手机                       880
搜索侠客行-首页                       252
付费流量(通过商家数字推广平台，灯火等投放的广告)      156
搜索侠客行-核心词                       84
搜索直达banner                      80
搜索邦道-销售线索                       69
公域搜索带券                          16
搜索-0元分期租手机                      10
搜索下挂券                            4
得劲租手机                            3
搜索-vivo                          2
搜索-租机                            2
搜索-租安卓手机                         1
搜索-租苹果耳机                         1
Name: count, dtype: int64

In [419]:
df[df['来源渠道']=='111'].value_counts()

Series([], Name: count, dtype: int64)

# 订单去重

In [282]:
# 保留不去重的数据
df_no_drop = df.copy()
# 订单去重
# to_do 京东不去重
df = clean.order_drop_duplicates(df)
df.shape


(118044, 78)

# 定义状态

In [283]:
# note 2025-02-24添加
df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
# , 'union_rent_tag', 'union_rent_rejected'
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
# df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
# df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
# note 2025-02-24修改
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)

In [284]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)


# 保留商家的数据

In [285]:
df_contain = df.copy()
df_contain.shape

(118044, 86)

# 剔除商家数据

In [286]:
# 调用Data_Clean类中的drop_merchant函数
df = clean.drop_merchant(df)

# 去重订单明细导出

In [287]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-09-08'

# 导入服务订单

In [288]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

df_j = df[df["进件"]=="进件"]
df_j.shape

(32884, 87)

In [289]:
df = clean.status_node(df)

# 导入台账数据

In [290]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023")  

f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
df_ck.shape

(33799, 18)

In [291]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")
dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape

(2674, 120)

In [292]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)
# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')


In [293]:
dfck.日期.value_counts().sort_index(ascending=False)

日期
2025-09-07    113
2025-09-06    103
2025-09-05    135
2025-09-04    150
2025-09-03    136
2025-09-02    154
2025-09-01    130
2025-08-31     71
2025-08-30     80
2025-08-29     98
2025-08-28    119
2025-08-27    131
2025-08-26    145
2025-08-25    149
2025-08-24    140
2025-08-23    123
2025-08-22    175
2025-08-21    158
2025-08-20    169
2025-08-19    128
Name: count, dtype: int64

# 每日各渠道出库单数

In [294]:
# 按转化日期看每日各渠道出库单数
df_weekday_zh = pd.crosstab(dfck["日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数
df_weekday = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
# 按下单日期看每日各渠道出库单数（纯租物）
df_ly_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
# 按下单日期看每日支付宝直播商品出库单数
df_zfb_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(dfck["下单日期"],dfck["押金类型"],margins=True)

In [295]:
# dfck['归属渠道'].unique()

# 总体转化

## 剔除拒量订单

In [296]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

In [297]:
# def year_sum(df, year):
#     df2 = df[df.年份==year]
#     df2_ck = df2[df2.是否出库==1]
#     df2_group = df2_ck.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
#     return df2_group
# df_group_2023 = year_sum(df, 2023)
# df_group_2024 = year_sum(df, 2024)
# df_group_2025 = year_sum(df, 2025)
# df_group_2025
# with pd.ExcelWriter(f'F:/需求/七月需求/23-25年出库数据1.xlsx', engine='xlsxwriter') as writer:
#     df_group_2023.to_excel(writer, sheet_name='2023年出库数据')
#     df_group_2024.to_excel(writer, sheet_name='2024年出库数据')
#     df_group_2025.to_excel(writer, sheet_name='2025年出库数据')

# df_new = df[~df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机'])]
# df_new_g = df_new[df_new.是否出库==1].groupby('下单月份').agg({'是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'new_actual_money': '签约价', '是否出库': '出库'})
# df_g = df.groupby('下单月份').agg({'是否进件': 'sum'}).rename(columns={'是否进件': '进件'})
# df_group = df_g.merge(df_new_g, on='下单月份', how='left').reset_index()
# df_group[['进件', '出库', '签约价']].set_index('进件')

In [298]:
# # 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# # 未完成订单的签约价-已收new_actual_money-rent_received
# # 剔除商家拒量的订单
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
# df_copy = df[['下单月份', 'status', '是否进件' ,'是否出库' ,'purchase_amount' ,'new_actual_money', '已收租金']]
# df_copy
# # 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
# df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([4,5]), df_copy['new_actual_money']-df_copy['已收租金'], 0)
# # 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
# df_res = df_copy.groupby('下单月份').agg({'是否进件': 'sum', '是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum', '应收未收金额': 'sum'}).rename(columns={'是否进件': '进件', '是否出库': '出库', 'purchase_amount': '采购价','new_actual_money': '签约价'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# # 写入excel
# # with pd.ExcelWriter(f'F:/需求/倩姐需求/业务量数据.xlsx', engine='openpyxl') as writer:
# #     df_res.to_excel(writer, sheet_name='业务量数据')

In [299]:
# df_241205.loc[:, '机审强拒_拒量'] = np.where(df_241205.qvt_risk=='0', 1, 0)
# df_241205[df_241205.机审强拒_拒量==1].shape


In [300]:
# 总体数据
# df = df[df.归属渠道!='京东渠道']
# df2 = df2[df2.归属渠道!='京东渠道'] 
df_all = all_models.data_group(df, df2, df_risk_examine, '下单日期')
df_all['出库（按下单时间）'] = df_all['出库']
df_all['出库（按转化时间）'] = df_weekday_zh['All'][:-1]
df_all = df_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）', '拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_all_new = df_all.reset_index()
df_all_new.insert(0, '星期', df_all_new['下单日期'].apply(lambda x: day_name_cn_mapping[x.day_name()]))
df_all_new.insert(1, '月份', df_all_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_all_new['下单日期'].astype(str).str.split('-').str[1])
df_all_new.tail()

,星期,月份,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
16,星期四,2025-09,2025-09-04,5829,2101,36.04%,1669,28.63%,1195,71.60%,474.0,256,218.0,13.06%,24,39,11.0,2,142,150.0,0,8.51%,8.51%,0.00%,1.44%,15.34%,0.66%,2.34%,2.44%
17,星期五,2025-09,2025-09-05,5616,1935,34.46%,1566,27.88%,1117,71.33%,449.0,238,211.0,13.47%,22,31,18.0,7,133,135.0,0,8.49%,8.49%,0.00%,1.40%,15.20%,1.15%,1.98%,2.37%
18,星期六,2025-09,2025-09-06,4788,1679,35.07%,1429,29.85%,1024,71.66%,406.0,218,188.0,13.16%,25,18,9.0,29,107,103.0,0,7.49%,7.49%,0.00%,1.75%,15.26%,0.63%,1.26%,2.23%
19,星期日,2025-09,2025-09-07,4611,1700,36.87%,1300,28.19%,945,72.69%,355.0,158,197.0,15.15%,26,24,9.0,44,94,113.0,0,7.23%,7.23%,0.00%,2.00%,12.15%,0.69%,1.85%,2.04%
20,星期一,2025-09,2025-09-08,2595,935,36.03%,746,28.75%,513,68.77%,233.0,60,173.0,23.19%,97,1,5.0,47,23,0.0,0,3.08%,3.08%,0.00%,13.00%,8.04%,0.67%,0.13%,0.89%


In [301]:
# df_all.to_excel(f'F:/需求/瑞彤需求/支付宝数据_{t_date}.xlsx')

# 搜索渠道转化

In [302]:
df_ss = df[df['归属渠道']=='搜索渠道']
df_ss2 = df2[df2['归属渠道']=='搜索渠道']
df_ss_group = all_models.data_group(df_ss, df_ss2, df_risk_examine, '下单日期')

# df_ss_group['出库（按下单时间）'] = df_weekday['搜索渠道'][:-1]
df_ss_group['出库（按下单时间）'] = df_ss_group['出库']
df_ss_group['出库（按转化时间）'] = df_weekday_zh['搜索渠道'][:-1]
df_ss_group = df_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系", "出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_ss_group_new = df_ss_group.reset_index()
df_ss_group.tail()


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,
2025-09-04,1364,124,9.09%,406,29.77%,277,68.23%,129.0,76,53.0,13.05%,3,13,0.0,0,37,39.0,9.11%,0.74%,18.72%,0.00%,3.20%,2.71%
2025-09-05,1329,112,8.43%,374,28.14%,231,61.76%,143.0,84,59.0,15.78%,3,5,0.0,1,50,53.0,13.37%,0.80%,22.46%,0.00%,1.34%,3.76%
2025-09-06,1170,116,9.91%,356,30.43%,229,64.33%,127.0,66,61.0,17.13%,10,7,1.0,10,33,30.0,9.27%,2.81%,18.54%,0.28%,1.97%,2.82%
2025-09-07,1117,98,8.77%,357,31.96%,263,73.67%,94.0,45,49.0,13.73%,5,2,1.0,13,28,36.0,7.84%,1.40%,12.61%,0.28%,0.56%,2.51%
2025-09-08,597,55,9.21%,173,28.98%,111,64.16%,62.0,21,41.0,23.70%,22,1,1.0,8,9,0.0,5.20%,12.72%,12.14%,0.58%,0.58%,1.51%


## 按周分组，搜索渠道非机审拒绝的订单

In [303]:
# df_ss = df[df['归属渠道']=='搜索渠道']
# # 将下单周转换为时间范围格式，例如"2025.8.20-2025.8.24"
# def get_week_range(date):
#     # 获取该日期所在周的周一
#     monday = date - pd.Timedelta(days=date.weekday())
#     # 获取该日期所在周的周日
#     sunday = monday + pd.Timedelta(days=6)
#     # 格式化为"年.月.日-年.月.日"格式
#     return f"{monday.year}.{monday.month}.{monday.day}-{sunday.year}.{sunday.month}.{sunday.day}"

# df_ss_quyy = df_ss[((df_ss['机审强拒'] == 0)&((df_ss['人审拒绝']==1)|(df_ss['客户取消']==1)|(df_ss['无法联系']==1)|(df_ss['出库前风控强拒']==1)))]
# # 按星期设置列：下单周，周一至周日为一周，未满一周按一周算，eg:8月20到8月24，9月1日到9月4日都算一周
# df_ss_quyy['下单周范围'] = df_ss_quyy['下单日期'].apply(get_week_range)

# df_ss_quyy['取消原因2'] = df_ss_quyy['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_ss_quyy['取消原因2'].drop_duplicates()
# df_ss_quyy_group = df_ss_quyy.groupby(['下单周范围', '取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_ss_quyy_group = df_ss_quyy_group.sort_values(['下单周范围', '取消数'], ascending=[True, False])
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消数']/(df_ss_quyy['是否进件'].sum())
# df_ss_quyy_group['取消率'] = df_ss_quyy_group['取消率'].apply(lambda x: format(x, '.2%'))
# # df_ss_quyy_group.to_excel('F:/需求/瑞彤需求/搜索非机审拒绝的取消原因.xlsx', sheet_name='订单取消原因')
# df_ss_quyy_group

In [304]:
# df_ss_jg = df_ss_quyy[df_ss_quyy.取消原因2=='不接受监管设备'][['下单日期','order_number','机型内存']]
# df_ss_jg.to_excel('F:/需求/瑞彤需求/搜索不接受监管设备.xlsx', sheet_name='订单明细',index=False)

# 单人聊天 转化

In [305]:
df_dr = df[df['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr2 = df2[df2['来源渠道']=='单人聊天会话中的小程序消息卡片（分享）']
df_dr_group = all_models.data_group(df_dr, df_dr2, df_risk_examine,  '下单日期')

# df_dr_group['出库（按下单时间）'] = df_weekday['单人聊天会话中的小程序消息卡片（分享）'][:-1]
df_dr_group['出库（按下单时间）'] = df_dr_group['出库']
try:
    df_dr_group['出库（按转化时间）'] = df_weekday_zh['单人聊天会话中的小程序消息卡片（分享）'][:-1]
except:
    df_dr_group['出库（按转化时间）'] = 0
df_dr_group = df_dr_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0)
df_dr_group_new = df_dr_group.reset_index()

In [306]:
df_dr_group_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
16,2025-09-04,226,39,17.26%,52,23.01%,33.0,63.46%,0,0,0.00%,0.00%
17,2025-09-05,263,34,12.93%,63,23.95%,42.0,66.67%,0,0,0.00%,0.00%
18,2025-09-06,202,32,15.84%,54,26.73%,38.0,70.37%,0,0,0.00%,0.00%
19,2025-09-07,171,28,16.37%,35,20.47%,25.0,71.43%,0,0,0.00%,0.00%
20,2025-09-08,108,11,10.19%,29,26.85%,17.0,58.62%,0,0,0.00%,0.00%


# 芝麻渠道转化

In [307]:
df_zm = df[df['归属渠道']=='芝麻租物']
df_zm2 = df2[df2['归属渠道']=='芝麻租物']
df_zm_group = all_models.data_group(df_zm, df_zm2, df_risk_examine, '下单日期')

# df_zm_group['出库（按下单时间）'] = df_weekday['芝麻租物'][:-1]
df_zm_group['出库（按下单时间）'] = df_zm_group['出库']
df_zm_group['出库（按转化时间）'] = df_weekday_zh['芝麻租物'][:-1]
df_zm_group = df_zm_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_zm_group_new = df_zm_group.reset_index()
df_zm_group.tail()

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,,
2025-09-04,2379,1303,54.77%,405,17.02%,218.0,53.83%,187.0,96.0,91.0,22.47%,11,17,0.0,2,61,63.0,15.06%,2.72%,23.70%,0.00%,4.20%,2.56%
2025-09-05,2221,1207,54.34%,374,16.84%,194.0,51.87%,180.0,95.0,85.0,22.73%,12,15,1.0,3,54,54.0,14.44%,3.21%,25.40%,0.27%,4.01%,2.43%
2025-09-06,1818,1006,55.34%,318,17.49%,155.0,48.74%,163.0,87.0,76.0,23.90%,12,8,0.0,9,47,43.0,14.78%,3.77%,27.36%,0.00%,2.52%,2.59%
2025-09-07,1861,1055,56.69%,322,17.30%,164.0,50.93%,158.0,62.0,96.0,29.81%,14,17,1.0,21,43,53.0,13.35%,4.35%,19.25%,0.31%,5.28%,2.31%
2025-09-08,1061,585,55.14%,194,18.28%,101.0,52.06%,93.0,15.0,78.0,40.21%,44,0,0.0,27,7,0.0,3.61%,22.68%,7.73%,0.00%,0.00%,0.66%


In [308]:
# df_zm_group_new

# 纯租物 转化

In [309]:
#逻辑是否需要优化   
df_zw = df[df['来源渠道']=='芝麻信用']
df_zw2 = df2[df2['来源渠道']=='芝麻信用']
df_zw_group = all_models.data_group(df_zw, df_zw2, df_risk_examine, '下单日期')

# df_zw_group['出库（按下单时间）'] = df_ly_ck['芝麻租物'][:-1]
df_zw_group['出库（按下单时间）'] = df_zw_group['出库']
df_zw_group = df_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例",'出库（按下单时间）','进件出库率','订单出库率']]
df_zw_group_new = df_zw_group.reset_index()
df_zw_group_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,进件出库率,订单出库率
16,2025-09-04,2334,1277,54.71%,400,17.14%,216.0,54.00%,61,15.25%,2.61%
17,2025-09-05,2164,1174,54.25%,369,17.05%,193.0,52.30%,53,14.36%,2.45%
18,2025-09-06,1810,1002,55.36%,317,17.51%,154.0,48.58%,47,14.83%,2.60%
19,2025-09-07,1849,1049,56.73%,321,17.36%,163.0,50.78%,43,13.40%,2.33%
20,2025-09-08,1054,582,55.22%,192,18.22%,99.0,51.56%,7,3.65%,0.66%


In [310]:
# df_zm.是否进件.sum()

# 抖音转化

In [311]:
df_dy = df[df['归属渠道']=='抖音渠道']
df_dy2 = df2[df2['归属渠道']=='抖音渠道']
# df_dy_group = all_models.data_group(df_dy, df_dy2, df_risk_examine, '下单日期')
# df_dy_group = clean.continuous_dates(df_dy_group)
# df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
# try:
#     df_dy_group['出库（按转化时间）'] = df_weekday_zh['抖音渠道'][:-1]
# except:
#     df_dy_group['出库（按转化时间）'] = 0
# df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）','进件出库率','订单出库率']].fillna(0) 
# df_dy_group_new = df_dy_group.reset_index()


In [312]:
classify_id_list = [125,126,127,185]
dfck.loc[:, '抖音电脑'] = np.where((dfck.归属渠道=='抖音渠道')&(dfck.classify_id.isin(classify_id_list)), 1, 0)
# 按转化日期看抖音电脑品类出库
dfck_dy_pc = pd.crosstab(index=dfck['日期'], columns=dfck['抖音电脑'],margins=True)
# 电脑
df_dy_classify = df_dy[df_dy.classify_id.isin(classify_id_list)]
df_d2y_classify = df_dy2[df_dy2.classify_id.isin(classify_id_list)]
df_dy_group = all_models.data_group(df_dy_classify, df_d2y_classify, df_risk_examine, '下单日期')
df_dy_group = clean.continuous_dates(df_dy_group)
df_dy_group['出库（按下单时间）'] = df_dy_group['出库']
try:
    df_dy_group['出库（按转化时间）'] = dfck_dy_pc[1][:-1]
except:
    df_dy_group['出库（按转化时间）'] = 0
df_dy_group = df_dy_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","客户取消","取消率","人审拒绝","人审拒绝率","待审核","出库前风控强拒",'出库（按下单时间）','出库（按转化时间）', '进件出库率','订单出库率']].fillna(0) 
df_dy_group_new = df_dy_group.reset_index()

In [313]:
df_dy_group_new.tail()

,index,去重订单数,进件前取消,进件前取消率,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,客户取消,取消率,人审拒绝,人审拒绝率,待审核,出库前风控强拒,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
16,2025-09-04,2.0,2.0,100.00%,0.0,0.00%,0.0,0.00%,0.0,nan%,0.0,0.0,nan%,0.0,nan%,0.0,0.0,0.0,0,nan%,0.00%
17,2025-09-05,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0
18,2025-09-06,1.0,1.0,100.00%,0.0,0.00%,0.0,0.00%,0.0,nan%,0.0,0.0,nan%,0.0,nan%,0.0,0.0,0.0,0,nan%,0.00%
19,2025-09-07,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0
20,2025-09-08,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0,0


# 支付宝直播转化

In [314]:
# df_zfb2[(df_zfb2['下单日期']=='2025-07-19')&(df_zfb2['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]
# 先被拒了，然后进入回捞再出库的，不是原先的出库

In [315]:
# df_zfb[(df_zfb['下单日期']=='2025-07-19')&(df_zfb['是否出库']==1)][['下单日期','order_number','activity_name','是否拒量','是否出库']]

In [316]:
# # 支付宝直播
df_zfb = df[df['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb2 = df2[df2['activity_name'].str.contains('支付宝直播商品')==True]
df_zfb_group = all_models.data_group(df_zfb, df_zfb2, df_risk_examine, '下单日期')
df_zfb_group = clean.continuous_dates(df_zfb_group)

# df_zfb_group['出库（按下单时间）'] = df_zfb_ck['支付宝直播商品'][:-1]
df_zfb_group['出库（按下单时间）'] = df_zfb_group['出库']
df_zfb_group['出库（按转化时间）'] = df_zfb_group['出库']
df_zfb_group = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）', '出库（按转化时间）','进件出库率','订单出库率']]
df_zfb_group1 = df_zfb_group[["去重订单数","进件前取消","进件前取消率","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","人审拒绝率","风控通过件","客户取消","取消率",'无法联系',"出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','订单出库率']].reset_index(names=['下单日期'])
df_zfb_group1.tail()

,下单日期,去重订单数,进件前取消,进件前取消率,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,客户取消,取消率,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,订单出库率
16,2025-09-04,69,34,49.28%,17,24.64%,18,26.09%,11.0,61.11%,7.0,3.0,16.67%,4.0,0,0.00%,0,3.0,0,1,5.56%,1.45%
17,2025-09-05,73,40,54.79%,20,27.40%,13,17.81%,8.0,61.54%,5.0,2.0,15.38%,3.0,1,7.69%,0,2.0,0,0,0.00%,0.00%
18,2025-09-06,81,45,55.56%,22,27.16%,14,17.28%,8.0,57.14%,6.0,6.0,42.86%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%
19,2025-09-07,72,40,55.56%,16,22.22%,16,22.22%,7.0,43.75%,9.0,2.0,12.50%,7.0,1,6.25%,0,0.0,1,5,31.25%,6.94%
20,2025-09-08,41,24,58.54%,9,21.95%,8,19.51%,7.0,87.50%,1.0,1.0,12.50%,0.0,0,0.00%,0,0.0,0,0,0.00%,0.00%


In [317]:
# df_zfb.下单日期.unique()

## 支付宝直播进件前取消原因

In [318]:
# df_zfb_group1.to_excel('F:\需求\崔婷玉需求\支付宝直播特定时间段转化数据.xlsx', index=False)
# df_zfb_group1.下单日期.unique()
# df_zfb_jjqqx = df_zfb[(df_zfb['进件前取消']==1)]
# df_zfb_jjqqx['取消原因2'] = df_zfb_jjqqx['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_zfb_jjqqx['取消原因2'].drop_duplicates()
# df_zfb_jjqqx_group = df_zfb_jjqqx.groupby(['取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消数']/(df_zfb_jjqqx_group['取消数'].sum())
# df_zfb_jjqqx_group['取消率'] = df_zfb_jjqqx_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_zfb_jjqqx_group.to_excel('F:/需求/崔婷玉需求/支付宝直播进件前订单取消原因2.xlsx', sheet_name='进件前订单取消原因')



In [319]:
df1_ck = df_zfb[df_zfb['是否出库']==1]
Today_zfb = datetime.now().strftime('%Y%m%d%H')

# 总体剔除直播数据

In [320]:
df_tc = df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dy_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]
df_tc.fillna(df_all[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_dr_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']]-df_zfb_group[['去重订单数', '进件数', '机审强拒', '出库（按下单时间）', '出库（按转化时间）']], inplace=True)
df_tc['预授权通过率'] = df_tc['进件数'] / df_tc['去重订单数']
df_tc['预授权通过率'] = df_tc['预授权通过率'].apply(lambda x: format(x, '.2%'))
df_tc['强拒比例'] = df_tc['机审强拒'] / df_tc['进件数']
df_tc['强拒比例'] = df_tc['强拒比例'].apply(lambda x: format(x, '.2%'))
df_tc['进件出库率'] = df_tc['出库（按下单时间）'] / df_tc['进件数']
df_tc['进件出库率'] = df_tc['进件出库率'].apply(lambda x: format(x, '.2%'))
df_tc['订单出库率'] = df_tc['出库（按下单时间）'] / df_tc['去重订单数']
df_tc['订单出库率'] = df_tc['订单出库率'].apply(lambda x: format(x, '.2%'))
df_tc = df_tc[['去重订单数', '进件数', '预授权通过率', '机审强拒', '强拒比例', '出库（按下单时间）', '出库（按转化时间）', '进件出库率', '订单出库率']].reset_index()
df_tc.tail()

,下单日期,去重订单数,进件数,预授权通过率,机审强拒,强拒比例,出库（按下单时间）,出库（按转化时间）,进件出库率,订单出库率
16,2025-09-04,5532.0,1599.0,28.90%,1151.0,71.98%,141.0,149.0,8.82%,2.55%
17,2025-09-05,5280.0,1490.0,28.22%,1067.0,71.61%,133.0,135.0,8.93%,2.52%
18,2025-09-06,4504.0,1361.0,30.22%,978.0,71.86%,107.0,103.0,7.86%,2.38%
19,2025-09-07,4368.0,1249.0,28.59%,913.0,73.10%,89.0,108.0,7.13%,2.04%
20,2025-09-08,2446.0,709.0,28.99%,489.0,68.97%,23.0,0.0,3.24%,0.94%


# 京享租 2025-05-20正式上线

In [321]:
df_no_drop_jd = df_no_drop[df_no_drop.归属渠道=='京东渠道']
df_no_drop_jd = df_no_drop_jd[df_no_drop_jd.下单日期>='2025-05-20']
df_no_drop_jd.loc[:, '创建进件数'] = np.where(df_no_drop_jd.进件=='进件', 1, 0)
df_no_drop_jd_g = df_no_drop_jd.groupby('下单日期').agg({'order_id': 'count', '创建进件数': 'sum'}).rename(columns={'order_id': '创建订单数'})
df_no_drop_jdsc_g = df_no_drop_jd[df_no_drop_jd.app_type=='JD_APP'].groupby('下单日期').agg({'order_id': 'count'}).rename(columns={'order_id': '创建订单数'})
# df_no_drop_jd_g.head()


In [322]:
df_jd = df[df['归属渠道']=='京东渠道']
df_jd2 = df2[df2['归属渠道']=='京东渠道']
df_jd = df_jd[df_jd.下单日期>='2025-05-20']
df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期')


df_jd_group['出库（按下单时间）'] = df_jd_group['出库']
try:
    df_jd_group['出库（按转化时间）'] = df_weekday_zh['京东渠道'][:-1]
except:
    df_jd_group['出库（按转化时间）'] = 0
df_jd_group = df_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','出库（按转化时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jd_group_new = df_jd_group.reset_index()
df_jd_group_new = df_no_drop_jd_g.merge(df_jd_group_new, on='下单日期', how='inner')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
16,2025-09-04,1715,757,1303,542,41.60%,705,54.11%,598.0,84.82%,107.0,49.0,58.0,8.23%,9,9,0.0,0,40,46.0,5.67%,1.28%,6.95%,0.00%,1.28%,3.07%
17,2025-09-05,1624,670,1211,499,41.21%,636,52.52%,552.0,86.79%,84.0,38.0,46.0,7.23%,6,11,0.0,2,27,26.0,4.25%,0.94%,5.97%,0.00%,1.73%,2.23%
18,2025-09-06,1413,634,1093,441,40.35%,604,55.26%,525.0,86.92%,79.0,41.0,38.0,6.29%,3,2,0.0,9,24,25.0,3.97%,0.50%,6.79%,0.00%,0.33%,2.20%
19,2025-09-07,1318,520,1001,449,44.86%,497,49.65%,421.0,84.71%,76.0,39.0,37.0,7.44%,3,5,0.0,8,21,23.0,4.23%,0.60%,7.85%,0.00%,1.01%,2.10%
20,2025-09-08,698,317,557,238,42.73%,305,54.76%,257.0,84.26%,48.0,12.0,36.0,11.80%,21,0,0.0,10,5,0.0,1.64%,6.89%,3.93%,0.00%,0.00%,0.90%


## 客户取消

In [323]:
# df_jd.columns

### 取消原因1：创建订单后处于订单取消状态

In [324]:
# df_no_drop_jd.loc[:, '取消1'] = np.where((df_no_drop_jd['status2'] == '订单取消')&(df_no_drop_jd['order_id'].notna()), 1, 0)
# df_no_drop_jd_group = df_no_drop_jd.groupby('下单月份').agg({'取消1':'sum', 'order_id':'count'}).rename(columns={'取消1':'取消数', 'order_id':'创建订单数'})
# df_no_drop_jd_group['取消率'] = (df_no_drop_jd_group['取消数']/df_no_drop_jd_group['创建订单数']).apply(lambda x: format(x, '.2%'))
# df_no_drop_jd_group

### 取消原因2：进件后，存在机审强拒、人审拒绝、客户取消、无法联系、出库前风控强拒中至少有一个为1

In [325]:
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd.loc[:, '取消2'] = np.where(((df_jd['机审强拒'] == 1)|(df_jd['人审拒绝']==1)|(df_jd['客户取消']==1)|(df_jd['无法联系']==1)|(df_jd['出库前风控强拒']==1)), 1, 0)
# df_jd_group = df_jd.groupby('下单月份').agg({'取消2':'sum', '是否进件':'sum'}).rename(columns={'取消2':'取消数', '是否进件':'进件数'})
# df_jd_group['取消率'] = (df_jd_group['取消数']/df_jd_group['进件数']).apply(lambda x: format(x, '.2%'))
# df_jd_group

### 取消原因4：审核通过后或有支付流水且状态为已退款

In [326]:
# df_jd.loc[:, '审核通过'] = np.where(df_jd['total_describes'].str.contains('msg=审核通过'), 1, 0)
# df_jd.loc[:, '取消4'] = np.where((df_jd['审核通过']==1)&(df_jd['status2']=='已退款'), 1, 0)
# df_jd_group_sh = df_jd.groupby('下单月份').agg({'取消4':'sum', '审核通过':'sum'}).rename(columns={'取消4':'取消数', '审核通过':'审核通过数'})
# df_jd_group_sh.loc[:, '取消率'] = (df_jd_group_sh['取消数']/df_jd_group_sh['审核通过数']).apply(lambda x: format(x, '.2%'))
# df_jd_group_sh

In [327]:
# 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消原因订单_月份.xlsx', engine='openpyxl') as writer:
#     df_no_drop_jd_group.to_excel(writer, sheet_name='取消原因1')
#     df_jd_group.to_excel(writer, sheet_name='取消原因2')
#     df_jd_group_sh.to_excel(writer, sheet_name='取消原因4')

### 发货前订单取消原因

In [328]:
# df_jd_shtg = df_jd#[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')]
# df_jd_shtg.loc[:,'发货前取消'] = np.where((df_jd['status2']=='已退款')&(df_jd['风控通过件']==1)&(df_jd['出库前风控强拒']==0),1,0)
# # 物流状态=5，退货拒签
# # df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
# df_jd_shtg.loc[:,'发货拒签'] = np.where((df_jd['物流状态']==5)&(df_jd['是否出库']==1),1,0)
# df_jd_shtg_group = df_jd_shtg.groupby('下单月份').agg({'发货前取消':'sum', '发货拒签':'sum', '是否进件':'sum'})#.rename(columns={'发货前取消':'发货前取消数', '发货拒签':'发货拒签数'})
# df_jd_shtg_group['发货前取消率'] = (df_jd_shtg_group['发货前取消']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group['发货拒签率'] = (df_jd_shtg_group['发货拒签']/df_jd_shtg_group['是否进件']).apply(lambda x: format(x, ".2%"))
# df_jd_shtg_group

In [329]:
# df_jd_shtg = df_jd[(df_jd['风控通过件']==1)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# df_jd_shtg = df_jd[(df_jd['是否进件'] == 1) & (df_jd['机审强拒'] == 0) & (df_jd['人审拒绝'] == 0)&(df_jd['status2']=='已退款')&(df_jd['出库前风控强拒']==0)]
# df_jd_shtg['取消原因2'] = df_jd_shtg['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
# df_jd_shtg['取消原因2'].drop_duplicates()
# df_jd_shtg_group = df_jd_shtg.groupby(['下单月份', '取消原因2']).agg({'order_number': 'count'}).rename(columns={'order_number': '取消数'})
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消数']/(df_jd_shtg['是否进件'].sum())
# df_jd_shtg_group['取消率'] = df_jd_shtg_group['取消率'].apply(lambda x: format(x, '.2%'))
# df_jd_shtg_group#.to_excel('F:/需求/瑞彤需求/京东发货前取消订单_月份.xlsx', sheet_name='审核通过发货前取消订单')

In [330]:
# # 写入Excel
# with pd.ExcelWriter('F:/需求/瑞彤需求/京东取消订单_月份.xlsx', engine='openpyxl') as writer:
#     df_tkjd_kh_group.to_excel(writer, sheet_name='客户取消')
#     df_jdqj_group.to_excel(writer, sheet_name='机审人审拒绝')
#     df_jd_shtg_group.to_excel(writer, sheet_name='审核通过取消订单')

## 京东商城

In [331]:
df_jdsc = df_jd[df_jd.app_type=='JD_APP']
df_jdsc2 = df_jd2[df_jd2.app_type=='JD_APP']
df_jdsc_group = all_models.data_group(df_jdsc, df_jdsc2, df_risk_examine, '下单日期')
df_jdsc_group['出库（按下单时间）'] = df_jdsc_group['出库']

df_jdsc_group = df_jdsc_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核",'出库（按下单时间）','进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_jdsc_group_new = df_jdsc_group.reset_index()
df_jdsc_group_new = df_no_drop_jdsc_g.merge(df_jdsc_group_new, on='下单日期', how='left')
df_jdsc_group_new.tail()

,下单日期,创建订单数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
16,2025-09-04,517,471,0,0.00%,458,97.24%,390.0,85.15%,68.0,34.0,34.0,7.42%,4,3,0.0,0,27,5.90%,0.87%,7.42%,0.00%,0.66%,5.73%
17,2025-09-05,478,442,0,0.00%,424,95.93%,374.0,88.21%,50.0,22.0,28.0,6.60%,4,4,0.0,0,20,4.72%,0.94%,5.19%,0.00%,0.94%,4.52%
18,2025-09-06,422,395,0,0.00%,383,96.96%,331.0,86.42%,52.0,29.0,23.0,6.01%,2,1,0.0,4,16,4.18%,0.52%,7.57%,0.00%,0.26%,4.05%
19,2025-09-07,353,321,0,0.00%,313,97.51%,266.0,84.98%,47.0,21.0,26.0,8.31%,1,3,0.0,7,15,4.79%,0.32%,6.71%,0.00%,0.96%,4.67%
20,2025-09-08,216,207,0,0.00%,202,97.58%,169.0,83.66%,33.0,8.0,25.0,12.38%,13,0,0.0,8,4,1.98%,6.44%,3.96%,0.00%,0.00%,1.93%


# 渠道的退货订单数

In [332]:
# 输出df归属渠道的种类
# df['归属渠道'].unique()
# 获取df中所以退货拒签的订单
df_qudao = df2[df2['是否出库']==1]
# df_qudao = df_qudao[df_qudao.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]#无用
# df_qudao = df_qudao[~df_qudao.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# 将归属渠道中的渠道数据依次取出芝麻租物， 搜索渠道，单人会话，S量，抖音渠道，，京东渠道（且前面六个渠道都需排除拒量），拒量(已存在是否拒量)的退货订单数
df_qudao['芝麻租物'] = np.where((df_qudao.归属渠道=='芝麻租物')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['搜索渠道'] = np.where((df_qudao.归属渠道=='搜索渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['单人会话'] = np.where((df_qudao.归属渠道=='单人会话')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
conditions_s = ['八派信息', 'CPS直播', '派金花', '九州信息', '勉丫租', '哈银', '租瓜直播', '美仑美奂',
                        '分期乐1', '分期乐2', '98租超', '推一推', '硬派抖音']
df_qudao['S量'] = np.where((df_qudao.归属渠道.isin(conditions_s))&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
df_qudao['抖音渠道'] = np.where((df_qudao.归属渠道=='抖音渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['拒量'] = np.where((df_qudao['是否拒量']==1)&(df_qudao.退货拒签==1), 1, 0)

df_qudao['京东渠道'] = np.where((df_qudao.归属渠道=='京东渠道')&(df_qudao.是否拒量!=1)&(df_qudao.退货拒签==1), 1, 0)
# 将各个渠道的订单数相加
df_qudao_group = df_qudao.groupby('下单日期').agg({'是否出库':'sum','退货拒签':'sum','芝麻租物': 'sum', '搜索渠道': 'sum', '单人会话': 'sum', 'S量': 'sum', '抖音渠道': 'sum', '拒量': 'sum', '京东渠道': 'sum'}).rename(columns={'是否出库': '总体出库订单数', '退货拒签': '拒收订单数'})
df_qudao_group = df_qudao_group.reset_index(names=['下单日期'])

df_qudao_group.tail()#.to_excel('F:\日报\日报\拒收.xlsx', index=False, header=True)

# df_qudao['下单日期'].unique()

,下单日期,总体出库订单数,拒收订单数,芝麻租物,搜索渠道,单人会话,S量,抖音渠道,拒量,京东渠道
16,2025-09-04,142,0,0,0,0,0,0,0,0
17,2025-09-05,133,0,0,0,0,0,0,0,0
18,2025-09-06,107,0,0,0,0,0,0,0,0
19,2025-09-07,94,0,0,0,0,0,0,0,0
20,2025-09-08,23,0,0,0,0,0,0,0,0


## 验证退货拒签的订单数是否正确

In [333]:
# 计算df_qudao_group所有下单日期的拒收订单数
# df_qudao_group['拒收订单数'].sum()


In [334]:
# sql_th = """
# select om.order_number,om.create_time
# from db_digua_business.t_order om
# left join db_digua_business.t_order_logistics tolog on om.id = tolog.order_id
# where tolog.status =  5
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-06-01'
# and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-06-30'
# """
# df_th = clean.query(sql_th)
# # 统计订单总数
# df_th


# 免审转化

判断免人审的单可以用t_order_risk表中的risk json串中的is_vip=1和status=0

In [335]:
df_ms = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2 = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_group = all_models.data_group(df_ms, df_ms2, df_risk_examine,  '下单日期')

df_ms_group = df_ms_group[["进件数","人审拒绝", "客户取消","出库","待审核","进件出库率","取消率","人审拒绝率"]] 
# df_ms_group = df_ms_group[["进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    # "出库前风控强拒","待审核",'出库', '进件出库率','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']]
df_ms_group_new = df_ms_group.reset_index()
df_ms_group.tail()


,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
下单日期,,,,,,,,
2025-09-04,448,218.0,24,140,2,31.25%,5.36%,48.66%
2025-09-05,406,185.0,21,133,6,32.76%,5.17%,45.57%
2025-09-06,376,170.0,25,107,27,28.46%,6.65%,45.21%
2025-09-07,354,138.0,24,94,41,26.55%,6.78%,38.98%
2025-09-08,212,39.0,89,23,47,10.85%,41.98%,18.40%


## 信审需求（倩姐）

In [336]:

# # df_ms_list = df_ms[(df_ms['下单日期']=='2025-08-18')&(df_ms['机审通过件']==1)]['order_id'].tolist()
# # df2[df2['机审通过件']==1].groupby('下单日期').agg({'order_id':'count'})
# # 免审订单号列表
# df_ms_list = df[(df['下单日期']=='2025-08-18')&(df['机审通过件']==1)&(df.is_vip=='1')&(df.status_result=='0')]['order_id'].tolist()
# len(df_ms_list)
# # df_ms_list
# # 机审通过件中缺少拒量的通过件，信审订单数
# # df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)&(~df2['order_id'].isin(df_ms_list))&((df2['is_vip'] == '0'))]['order_id'].count()
# # 机审通过件，不含拒量的通过件
# df2[(df2['下单日期']=='2025-08-18')&(df2['机审通过件']==1)]['order_id'].count()

### 获取机审通过订单中拒量的订单

In [337]:
# def xmy(df, df_risk_examine):
#         '''
#         计算小蚂蚁租机的总体转移，机审强拒，人审拒绝，出库前风控强拒数据
#         :param df: 总体数据的df
#         :param df_risk_examine: 出库前风控强拒数据的df
#         :param model: 传入的分组参数
#         :return: 返回小蚂蚁租机的数据
#         '''
#         # 匹配小蚂蚁（拒绝）数据
#         df_xmy = df[df['merchant_name'].isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
#         # 出库前强拒数据重命名
#         df_risk_examine.rename(columns={'time': 'time_risk_ex', 'status': 'status_risk_ex'}, inplace=True)
#         # 对小蚂蚁数据和出库前强拒数据进行拼接
#         df_risk_examine_all = pd.merge(df_xmy, df_risk_examine, left_on='id_card_num', right_on='id_card', how='inner')
#         # 计算出库前强拒的订单数
#         df_risk_examine_all2 = df_risk_examine_all[(df_risk_examine_all['time_risk_ex'] < df_risk_examine_all['update_time'])]
#         # 进行排序并取到最近的一个订单
#         df_risk_examine_all2 = df_risk_examine_all2.sort_values(['order_id', 'time_risk_ex'], ascending=[True, False]).groupby('order_id').head(1)
#         df_risk_examine_all2 = df_risk_examine_all2[df_risk_examine_all2['status_risk_ex'] == '1']
#         # df_risk_examine_all2_g = df_risk_examine_all2.groupby(model).agg(order_risk_ex=('order_id', 'size'))
#         # df_risk_examine_all2_g
#         # 排除出库前风控强拒的订单
#         df_xmy_new = df_xmy[~df_xmy['order_id'].isin(df_risk_examine_all2['order_id'].to_list())]
#         # 定义人审拒绝订单
#         df_xmy_new['小蚂蚁人审拒绝'] = np.where(df_xmy_new['reason'] != '系统风控拒绝转移', 1, 0)
#         print(" 小蚂蚁人审拒绝：",df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].to_list())
#         print(df_xmy_new[(df_xmy_new['下单日期']=='2025-08-18')&(df_xmy_new['小蚂蚁人审拒绝']==1)]['order_id'].count())
#         print("order_risk_ex:", df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].to_list())
#         print(df_risk_examine_all2[(df_risk_examine_all2['下单日期']=='2025-08-18')]['order_id'].count())
        
# xmy(df, df_risk_examine)
# df_241205 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
# # 拒绝
# df_241205 = df_241205[~df_241205.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机', '喜卓灵新租机'])]
# df_241205.loc[:, '出库前强拒_拒量'] = np.where(((df_241205.qvt_risk == '0')|(df_241205.qvt_risk.isna()))&(df_241205.qvt_result == '1'), 1, 0)
# df_241205[(df_241205['出库前强拒_拒量']==1)&(df_241205['下单日期']=='2025-08-18')]['order_id'].count()


### 信审订单，按下单日期统计

In [338]:
# # 信审订单数
# df_credit = df2[(df2['机审通过件']==1)&((df2['is_vip'] == '0')&(df.status_result=='0'))]#  | (df['is_vip'].isna())
# # 筛选未命中发货需上锁的订单
# # df_credit_examine = df_credit_examine[~(df_credit_examine.tips.str.contains('发货需上锁', na=False))]
# # df_credit = df_credit[~(df_credit.tips.str.contains('发货需上锁', na=False))]
# # # 基于下单日期和id_card_num去重
# df_credit = df_credit.drop_duplicates(subset=['下单日期', 'id_card_num'],keep="first")
# df_credit_group = df_credit.groupby('下单日期').agg({'order_id':'count'})
# # df_creadit_list = df_credit[df_credit['下单日期']=='2025-08-18']['order_id'].tolist()
# # df_credit.tail()
# # len(df_creadit_list)
# df_credit_group

## 各渠道每日免审出库单数统计

In [339]:
# 剔除据量的数据
# dfck_ms = dfck[(~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)&(dfck.status!=12)]
dfck_ms = clean.drop_rejected_merchant(dfck)
dfdfck=dfck_ms[(dfck_ms.is_vip=='1')&(dfck_ms.status_result=='0')]

dfdfck_ck = pd.crosstab(dfdfck["下单日期"],dfdfck["归属渠道"],margins=True)[['芝麻租物','搜索渠道']][:-1]

# 删除原先索引
# dfdfck_ck = dfdfck_ck.reset_index(drop=True)
dfdfck_ck.rename(columns={'芝麻租物': '芝麻租物免审出库', '搜索渠道': '搜索免审出库'}, inplace=True)
dfdfck_ck = clean.continuous_dates(dfdfck_ck)
dfdfck_ck = dfdfck_ck.reset_index(names=['下单日期'])[['芝麻租物免审出库', '搜索免审出库']]#.set_index('下单日期')
dfdfck_ck.tail()


归属渠道,芝麻租物免审出库,搜索免审出库
16,61.0,37.0
17,54.0,50.0
18,44.0,32.0
19,40.0,25.0
20,0.0,0.0


## 租物  搜索的免审进件

In [340]:
dfms = df[(df.is_vip=='1')&(df.status_result=='0')]
dfms_jj = pd.crosstab(dfms["下单日期"],dfms["归属渠道"])
# 删除原先索引
dfms_jj = dfms_jj.reset_index(drop=True)
# 添加下单日期
dfms_jj['下单日期'] = df_ms_group_new['下单日期']
dfms_jj = dfms_jj[['下单日期', "芝麻租物","搜索渠道"]]
dfms_jj.rename(columns={'芝麻租物': '芝麻租物免审进件', '搜索渠道': '搜索渠道免审进件'}, inplace=True)
dfms_jj.tail()

归属渠道,下单日期,芝麻租物免审进件,搜索渠道免审进件
16,2025-09-04,207,126
17,2025-09-05,199,120
18,2025-09-06,181,114
19,2025-09-07,180,94
20,2025-09-08,103,52


In [341]:
# 整合
df_ms_new = dfms_jj.copy()
df_ms_new.insert(0, '月份', df_ms_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_new['下单日期'].astype(str).str.split('-').str[1])
df_ms_new.insert(2, '总体进件', df_all_new['进件数'])
df_ms_new.insert(3, '芝麻租物进件', df_zm_group_new['进件数'])
df_ms_new.insert(4, '免审进件', df_ms_group_new['进件数'])
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件'] / df_ms_new['总体进件']
df_ms_new.loc[:, '免审进件占比'] = df_ms_new['免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件'] / df_ms_new['芝麻租物进件']
df_ms_new.loc[:, '芝麻租物免审进件占比'] = df_ms_new['芝麻租物免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索渠道免审进件'] / df_ss_group_new['进件数']
df_ms_new.loc[:, '搜索免审进件占比'] = df_ms_new['搜索免审进件占比'].apply(lambda x: format(x, '.2%'))
df_ms_new.loc[:, '总体出库'] = df_all_new['出库（按下单时间）']
df_ms_new.loc[:, '芝麻租物出库'] = df_zm_group_new['出库（按下单时间）']
df_ms_new.loc[:, '免审出库'] = df_ms_group_new['出库']
df_ms_new2 = pd.concat([df_ms_new, dfdfck_ck], axis=1)
# df_ms_new2
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审出库'] / df_ms_new2['免审进件']
df_ms_new2.loc[:, '免审转化率'] = df_ms_new2['免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库'] / df_ms_new2['总体出库']
df_ms_new2.loc[:, '免审出库占比'] = df_ms_new2['免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库'] / df_ms_new2['芝麻租物出库']
df_ms_new2.loc[:, '芝麻租物免审出库占比'] = df_ms_new2['芝麻租物免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库'] / df_ss_group_new['出库（按下单时间）']
df_ms_new2.loc[:, '搜索免审出库占比'] = df_ms_new2['搜索免审出库占比'].apply(lambda x: format(x, '.2%'))
df_ms_new2.loc[:, '机审通过'] = df_all_new['机审通过件']
df_ms_new2.loc[:, '非免审进件'] = df_ms_new2['机审通过'] - df_ms_new2['免审进件']
df_ms_new2.loc[:, '非免审出库'] = df_ms_new2['总体出库'] - df_ms_new2['免审出库']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审出库'] / df_ms_new2['非免审进件']
df_ms_new2.loc[:, '非免审转化率'] = df_ms_new2['非免审转化率'].apply(lambda x: format(x, '.2%'))
df_ms_new3 = pd.concat([df_ms_new2, df_ms_group_new], axis=1)
# df_ms_new3.set_index('月份')
df_ms_new3.tail()

,月份,下单日期,总体进件,芝麻租物进件,免审进件,芝麻租物免审进件,搜索渠道免审进件,免审进件占比,芝麻租物免审进件占比,搜索免审进件占比,总体出库,芝麻租物出库,免审出库,芝麻租物免审出库,搜索免审出库,免审转化率,免审出库占比,芝麻租物免审出库占比,搜索免审出库占比,机审通过,非免审进件,非免审出库,非免审转化率,下单日期,进件数,人审拒绝,客户取消,出库,待审核,进件出库率,取消率,人审拒绝率
16,2025-09,2025-09-04,1669,405,448,207,126,26.84%,51.11%,31.03%,142,61,140,61.0,37.0,31.25%,98.59%,100.00%,100.00%,474.0,26.0,2,7.69%,2025-09-04,448,218.0,24,140,2,31.25%,5.36%,48.66%
17,2025-09,2025-09-05,1566,374,406,199,120,25.93%,53.21%,32.09%,133,54,133,54.0,50.0,32.76%,100.00%,100.00%,100.00%,449.0,43.0,0,0.00%,2025-09-05,406,185.0,21,133,6,32.76%,5.17%,45.57%
18,2025-09,2025-09-06,1429,318,376,181,114,26.31%,56.92%,32.02%,107,47,107,44.0,32.0,28.46%,100.00%,93.62%,96.97%,406.0,30.0,0,0.00%,2025-09-06,376,170.0,25,107,27,28.46%,6.65%,45.21%
19,2025-09,2025-09-07,1300,322,354,180,94,27.23%,55.90%,26.33%,94,43,94,40.0,25.0,26.55%,100.00%,93.02%,89.29%,355.0,1.0,0,0.00%,2025-09-07,354,138.0,24,94,41,26.55%,6.78%,38.98%
20,2025-09,2025-09-08,746,194,212,103,52,28.42%,53.09%,30.06%,23,7,23,0.0,0.0,10.85%,100.00%,0.00%,0.00%,233.0,21.0,0,0.00%,2025-09-08,212,39.0,89,23,47,10.85%,41.98%,18.40%


## 免审订单转化

In [342]:
# 免审
df_ms1_o = df[(df.is_vip=='1')&(df.status_result=='0')]
df_ms2_o = df2[(df2.is_vip=='1')&(df2.status_result=='0')]
df_ms_o_group = all_models.data_group(df_ms1_o, df_ms2_o, df_risk_examine, '下单日期')
df_ms_o_group = df_ms_o_group[["进件数","人审拒绝","客户取消","出库","待审核","进件出库率","取消率","人审拒绝率",'出库前风控强拒','出库前强拒比例','无法联系','无法联系占比']]
df_ms_o_group_new = df_ms_o_group.reset_index()
df_ms_o_group_new.insert(0, '月份', df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[0]+'-'+df_ms_o_group_new['下单日期'].astype(str).str.split('-').str[1])
# df_ms_o_group_new#.set_index('月份')

#  免押订单转化率 全免押

In [343]:
sql_ysq = '''
    select om.create_time, tmu.id_card_num,top.order_id,om.status
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )
    and om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    -- and top.total_freeze_fund_amount is not null
    -- and trade_no is not null
    '''
df_ysq = clean.query(sql_ysq)
df_ysq.shape

(161460, 5)

In [344]:
df_ysq_t = df_ysq.copy()
df_ysq_t["下单日期"]=df_ysq_t["create_time"].dt.date
df_ysq_t["下单日期"]=pd.to_datetime(df_ysq_t["下单日期"],errors="coerce")
df_ysq_t['是否前置拦截'] = np.where((df_ysq_t.result.str.contains('id_card不得为空') )|(df_ysq_t.result.str.contains('mobile校验不通过')) |(df_ysq_t.result.str.contains('name校验不通过'))
                        |(df_ysq_t.result.str.contains('年龄超过49岁或低于18岁'))|(df_ysq_t.result=='风控未通过')|(df_ysq_t.result=='黑名单用户'),1,0)
df_ysq_t.loc[:,"进件"]=np.where((df_ysq_t["status"]==1)|(df_ysq_t["status"]==13),"未进件","进件")
# 去除是否前置拦截=1且进件=未进件的行
df_ans = df_ysq_t[(df_ysq_t['是否前置拦截']==0)|(df_ysq_t['进件']=='进件')]
# 按下单日期与id_card_num去重
df_ans = df_ans.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
# 按下单日期分组计算订单数
df_ans = df_ans.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "预授权调起人数"}).reset_index()
# df_ans

In [345]:

sql_ysqmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )
    and om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount = 0
    and trade_no is not null
    and top.status in (2, 5)
    '''

df_ysqmy = clean.query(sql_ysqmy)
df_ysqmy["下单日期"]=df_ysqmy["create_time"].dt.date
df_ysqmy["下单日期"]=pd.to_datetime(df_ysqmy["下单日期"],errors="coerce")
df_ysqmy = df_ysqmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqmy = df_ysqmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "全免押"}).reset_index()
# df_ysqmy

In [346]:
df_qmy = df[df['押金类型']=='全免押']
df_qmy2 = df2[df2['押金类型']=='全免押']
df_qmy_group = all_models.data_group(df_qmy, df_qmy2, df_risk_examine, '下单日期')
df_qmy_group['出库（按下单时间）'] = df_my['全免押'][:-1]
df_qmy_group = df_qmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","待审核", '出库（按下单时间）', '进件出库率', '取消率', '人审转化率', '人审拒绝率']].fillna(0)
df_qmy_group_new = df_qmy_group.reset_index()
df_qmy_group_new.insert(1, '总进件', df_all_new['进件数'])
# df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_qmy_group_new.insert(2, '免押进件预授权占比', (df_ysqmy.全免押 / df_ans.预授权调起人数).apply(lambda x: format(x, '.2%') if pd.notna(x) else None))
# df_qmy_group_new.set_index('下单日期')
df_qmy_group_new.tail()

,下单日期,总进件,免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库（按下单时间）,进件出库率,取消率,人审转化率,人审拒绝率
16,2025-09-04,1669,30.52%,1617,1144,70.75%,473.0,255,218.0,13.48%,24,2,142.0,8.78%,1.48%,30.02%,15.77%
17,2025-09-05,1566,29.08%,1495,1047,70.03%,448.0,238,210.0,14.05%,22,7,131.0,8.83%,1.47%,29.46%,15.92%
18,2025-09-06,1429,31.04%,1380,975,70.65%,406.0,218,188.0,13.62%,25,29,102.0,7.75%,1.81%,26.35%,15.80%
19,2025-09-07,1300,31.90%,1251,897,71.70%,354.0,158,196.0,15.67%,26,44,86.0,7.43%,2.08%,26.27%,12.63%
20,2025-09-08,746,31.12%,721,489,67.82%,232.0,60,172.0,23.86%,97,47,0.0,3.19%,13.45%,9.91%,8.32%


# 非免押（部分免押）

In [347]:
sql_ysqfmy = '''
    select om.create_time, tmu.id_card_num,top.order_id
    from db_digua_business.t_order_pay top
    left join db_digua_business.t_order om on top.order_id = om.id
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id 
    where top.pay_type = 'ZFBYSQ' 
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-01-01'
    -- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-08-18'
    and om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )
    and om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')   
    and top.total_freeze_fund_amount > 0
    and trade_no is not null
    and top.status in (2, 5)
    '''
    
df_ysqfmy = clean.query(sql_ysqfmy)

df_ysqfmy["下单日期"]=df_ysqfmy["create_time"].dt.date
df_ysqfmy["下单日期"]=pd.to_datetime(df_ysqfmy["下单日期"],errors="coerce")
df_ysqfmy = df_ysqfmy.drop_duplicates(subset=["下单日期","id_card_num"],keep="first")
df_ysqfmy = df_ysqfmy.groupby(["下单日期"]).agg({"order_id": 'count'}).rename(columns={"order_id": "部分免押"}).reset_index()
# df_ysqfmy

In [348]:
df_fmy = df[df['押金类型']=='部分免押']
df_fmy2 = df2[df2['押金类型']=='部分免押']
df_fmy_group = all_models.data_group(df_fmy, df_fmy2, df_risk_examine, '下单日期')
df_fmy_group['待审核'] = 0
df_fmy_group = df_fmy_group[["进件数","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消", '待审核']]
try:
    df_fmy_group.loc[:, '出库'] = df_my['部分免押'][:-1]
except:
    df_fmy_group.loc[:, '出库'] = 0
df_fmy_group_new = df_fmy_group.reset_index()
df_fmy_group_new.insert(1, '总进件', df_all_new['进件数'])
df_fmy_group_new.insert(2, '非免押进件预授权占比', (df_ysqfmy.部分免押/df_ans.预授权调起人数).map(lambda x: format(x, '.2%')))
df_fmy_group_new.loc[:, '进件出库率'] = (df_fmy_group_new.出库/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '取消比例'] = (df_fmy_group_new.客户取消/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审转化率'] = (df_fmy_group_new.出库/df_fmy_group_new.机审通过件).map(lambda x: format(x, '.2%'))
df_fmy_group_new.loc[:, '人审拒绝率'] = (df_fmy_group_new.人审拒绝/df_fmy_group_new.进件数).map(lambda x: format(x, '.2%'))
df_fmy_group_new.tail()
# df_fmy_group_new#.set_index('下单日期')

,下单日期,总进件,非免押进件预授权占比,进件数,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,待审核,出库,进件出库率,取消比例,人审转化率,人审拒绝率
16,2025-09-04,1669,1.67%,52,51.0,98.08%,1.0,1,0.0,0.00%,0,0,0.0,0.00%,0.00%,0.00%,1.92%
17,2025-09-05,1566,2.39%,71,70.0,98.59%,1.0,0,1.0,1.41%,0,0,1.0,1.41%,0.00%,100.00%,0.00%
18,2025-09-06,1429,1.93%,49,49.0,100.00%,0.0,0,0.0,0.00%,0,0,0.0,0.00%,0.00%,nan%,0.00%
19,2025-09-07,1300,2.03%,49,48.0,97.96%,1.0,0,1.0,2.04%,0,0,1.0,2.04%,0.00%,100.00%,0.00%
20,2025-09-08,746,1.79%,25,24.0,96.00%,1.0,0,1.0,4.00%,0,0,NaN,nan%,0.00%,nan%,0.00%


In [349]:
# 将df_qmy_group_new，df_fmy_group_new写入excel中
# path = 'F:/需求/倩姐需求/'
# with pd.ExcelWriter(path + '预授权.xlsx') as writer:
#     df_qmy_group_new.to_excel(writer, sheet_name='免押', startrow=0, index=True)
#     df_fmy_group_new.to_excel(writer, sheet_name='非免押', startrow=0, index=True)

# 取消拒绝原因

In [350]:
# # 惜仪需求
# path_result = 'F:/myfile/数据提取及分析/'
# excel_name = f'取消&拒绝原因_{t_date}.xlsx'
# dfms = df2
# # 取消原因 总体
# qx_zt_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# # 拒绝原因总体
# jj_zt_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的电审拒绝原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# jj_ms_df = pd.crosstab(dfms["下单日期"],dfms["电审拒绝原因"],margins=True)
# #免审进件的取消原因
# dfms=df2[(df2["decision_result"].str.contains(pat="免人审",regex=False)==True)&(df2.is_vip=='1')&(df2.status_result=='0')]
# qx_ms_df = pd.crosstab(dfms["下单日期"],dfms["取消原因"],margins=True)

# with pd.ExcelWriter(path_result + excel_name, engine='xlsxwriter') as writer:  
#     qx_zt_df.to_excel(writer, sheet_name='取消原因-总体', startrow=0, index=True)
#     jj_zt_df.to_excel(writer, sheet_name='拒绝原因-总体', startrow=0, index=True) 
#     jj_ms_df.to_excel(writer, sheet_name='拒绝原因-免审', startrow=0, index=True) 
#     qx_ms_df.to_excel(writer, sheet_name='取消原因-免审', startrow=0, index=True) 

## 取消原因2&拒绝理由

In [351]:
# from datetime import datetime, timedelta
# # 获取昨日时间并转化为字符串
# from datetime import datetime  

# df2['取消原因2'] = df2['cancel_reason'].str.split('：')
# df2['取消原因2'] = df2['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
# y_date = datetime.today() - timedelta(days=1)

# y_date = y_date.strftime('%Y-%m-%d')
# df2_new = df2[(df2['下单日期'] == y_date)&(df2['拒绝理由'].notna())]
# # 切割拒绝理由内容，取第一个逗号前的内容
# # df2_new['拒绝理由'] = df2_new['拒绝理由'].str.split(',').str[0]
# df_quju = df2_new
# # 取消原因_总体
# qx_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["取消原因2"],margins=False)
# # 拒绝理由_总体
# jj_zt_df = pd.crosstab(df_quju["下单日期"],df_quju["拒绝理由"],margins=False)

# # jj_zt_df[jj_zt_df['下单日期'] == '2025-07-15']

# qx_zt_df
# df_quju.groupby('取消原因2').agg({'order_id': 'count'}).sort_values(by='order_id', ascending=False).rename(columns={'order_id': '订单数'})


## 京东订单取消原因

# 商家转化数据

In [352]:
df_contain["待审核"] = np.where(df_contain["审核状态"]=='待审核',1,0)
df_contain["前置拦截"] = np.where(df_contain["审核状态"]=='前置拦截',1,0)
df_contain["机审强拒"] = np.where(df_contain["审核状态"]=='机审强拒',1,0)
df_contain["人审拒绝"] = np.where(df_contain["审核状态"]=='人审拒绝',1,0)
df_contain["客户取消"] = np.where(df_contain["审核状态"]=='客户取消',1,0)
df_contain["出库前风控强拒"] = np.where(df_contain["审核状态"]=='出库前风控强拒',1,0)
df_contain["无法联系"] = np.where(df_contain["审核状态"]=='无法联系',1,0)
df_contain["是否进件"] = np.where(df_contain["进件"]=='进件',1,0)
df_contain["是否出库"] = np.where(df_contain["status"].isin([2,3,4,5,6,8,15]),1,0)

df_contain["进件前取消"] = np.where(df_contain["审核状态"]=='进件前取消',1,0)

## 澄心优租

In [353]:
# 澄心优租
cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
cxyz = clean.continuous_dates(cxyz)
cxyz_new = cxyz.reset_index(names=['下单日期'])

# try:
#     cxyz = all_models.merchant_names(df_contain, '澄心优租', '下单日期')
#     if cxyz.empty:
#         print("warn: 没有数据")
#         cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
#     else:
#         cxyz = cxyz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","订单出库率"]]
#         cxyz = clean.continuous_dates(cxyz)
#         cxyz_new = cxyz.reset_index(names=['下单日期'])
# except:
#     cxyz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','人审拒绝','人审拒绝率','待审核','出库','进件出库率','订单出库率'])
cxyz_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,人审拒绝,人审拒绝率,待审核,出库,进件出库率,订单出库率
16,2025-09-04,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0
17,2025-09-05,1.0,0.0,0.00%,1.0,100.00%,1.0,1.0,100.00%,0.0,0.0,0.00%,0.00%
18,2025-09-06,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%
19,2025-09-07,3.0,0.0,0.00%,0.0,0.00%,0.0,0.0,nan%,0.0,0.0,nan%,0.00%
20,2025-09-08,0.0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0


## 优优2店

In [354]:
# 优优2店
# yy2 = all_models.merchant_names(df_contain,'优优2店', '下单日期')
# yy2 = yy2[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yy2


## 北京海鸟窝科技有限公司

In [355]:
# 北京海鸟窝科技有限公司
hnw = all_models.merchant_names(df_contain,'北京海鸟窝科技有限公司', '下单日期')
hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
hnw = clean.continuous_dates(hnw)
hnw_new = hnw.reset_index(names=['下单日期'])
# 修改后更具有健壮性
# try:
#     hnw = all_models.merchant_names(df_contain, '北京海鸟窝科技有限公司', '下单日期')
#     if hnw.empty:
#         print("warn: 没有数据")
#         hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
#     else:
#         hnw = hnw[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
#         hnw = clean.continuous_dates(hnw)
#         hnw_new = hnw.reset_index(names=['下单日期'])
# except:
#     hnw_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
hnw_new.tail()


,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
16,2025-09-04,1.0,1.0,100.00%,0.0,0.00%
17,2025-09-05,4.0,1.0,25.00%,0.0,0.00%
18,2025-09-06,1.0,0.0,0.00%,0.0,nan%
19,2025-09-07,1.0,0.0,0.00%,0.0,nan%
20,2025-09-08,0.0,0.0,0,0.0,0


## 租着用电脑数码

In [356]:
# 租着用电脑数码
# zzy = all_models.merchant_names(df_contain,'租着用电脑数码', '下单日期')
# zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
# zzy = clean.continuous_dates(zzy)
# zzy_new = zzy.reset_index(names=['下单日期'])

try:
    zzy = all_models.merchant_names(df_contain, '租着用电脑数码', '下单日期')
    if zzy.empty:
        print("warn: 没有数据")
        zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
    else:
        zzy = zzy[["去重订单数","进件数","预授权通过率","出库","进件出库率"]]
        zzy = clean.continuous_dates(zzy)
        zzy_new = zzy.reset_index(names=['下单日期'])
except:
    zzy_new = pd.DataFrame(columns=['去重订单数','进件数','预授权通过率','出库','进件出库率'])
zzy_new.tail()

,下单日期,去重订单数,进件数,预授权通过率,出库,进件出库率
16,2025-09-04,9,1,11.11%,0,0.00%
17,2025-09-05,10,1,10.00%,0,0.00%
18,2025-09-06,8,0,0.00%,0,nan%
19,2025-09-07,14,1,7.14%,0,0.00%
20,2025-09-08,5,0,0.00%,0,nan%


## 格木木二奢名品

In [357]:
# 格木木二奢名品
gmm = all_models.merchant_names(df_contain,'格木木二奢名品', '下单日期')
gmm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 广州康基贸易有限公司

In [358]:
# 广州康基贸易有限公司
kjmy = all_models.merchant_names(df_contain,'广州康基贸易有限公司', '下单日期')
kjmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]


,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
下单日期,,,,,,,,,,,,,,


## 趣智数码

In [359]:
# 趣智数码
# qzsm = all_models.merchant_names(df_contain,'趣智数码', '下单日期')
# qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# qzsm = clean.continuous_dates(qzsm)
# qzsm_new = qzsm.reset_index(names=['下单日期'])

try:
    qzsm = all_models.merchant_names(df_contain, '趣智数码', '下单日期')
    if qzsm.empty:
        print("warn: 没有数据")
        qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        qzsm = qzsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        qzsm = clean.continuous_dates(qzsm)
        qzsm_new = qzsm.reset_index(names=['下单日期'])
except:
    qzsm_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
qzsm_new.tail()


,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,3.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
17,2025-09-05,2.0,0.0,0.00%,1.0,50.00%,1.0,0.0,0.0,0.00%,1.0,0.0,0.00%,0.00%,0.00%
18,2025-09-06,3.0,0.0,0.00%,1.0,33.33%,1.0,0.0,0.0,0.00%,1.0,0.0,0.00%,0.00%,0.00%
19,2025-09-07,5.0,0.0,0.00%,4.0,80.00%,4.0,3.0,0.0,0.00%,1.0,0.0,0.00%,75.00%,0.00%
20,2025-09-08,2.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%


## 汇客好租

In [360]:
# # 汇客好租
# hkhz = all_models.merchant_names(df_contain,'汇客好租', '下单日期')
# hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz = clean.continuous_dates(hkhz)
# hkhz_new = hkhz.reset_index(names=['下单日期'])
# # 电脑
# df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
# hkhz_dy = all_models.merchant_names(df_contain_dy,'汇客好租', '下单日期')
# hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# hkhz_dy = clean.continuous_dates(hkhz_dy)
# hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])


try:
    # 汇客好租
    hkhz = all_models.merchant_names(df_contain, '汇客好租', '下单日期')
    if hkhz.empty:
        print("warn: 没有数据")
        hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz = hkhz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz = clean.continuous_dates(hkhz)
        hkhz_new = hkhz.reset_index(names=['下单日期'])
except:
    hkhz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

try:
    # 电脑
    df_contain_dy = df_contain[(df_contain.归属渠道=='抖音渠道')&(df_contain.classify_id.isin([125,126,127,185]))]
    hkhz_dy = all_models.merchant_names(df_contain_dy, '汇客好租', '下单日期')
    if hkhz_dy.empty:
        print("warn: 没有数据")
        hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        hkhz_dy = hkhz_dy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        hkhz_dy = clean.continuous_dates(hkhz_dy)
        hkhz_dy_new = hkhz_dy.reset_index(names=['下单日期'])
except:
    hkhz_dy_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])

hkhz_new.tail(), hkhz_dy_new.tail()

(         下单日期  去重订单数  前置拦截     拦截率  进件数  预授权通过率  机审通过件  客户取消  人审拒绝   人审拒绝率  \
 16 2025-09-04     76     6   7.89%   38  50.00%     38     0    20  52.63%   
 17 2025-09-05     68    10  14.71%   37  54.41%     37     0    21  56.76%   
 18 2025-09-06     66     3   4.55%   42  63.64%     42     4    25  59.52%   
 19 2025-09-07     21     2   9.52%   14  66.67%     14     0     6  42.86%   
 20 2025-09-08     40     3   7.50%   21  52.50%     21     7    12  57.14%   
 
     待审核  出库   进件出库率     取消率  订单出库率  
 16    0   0   0.00%   0.00%  0.00%  
 17    0   2   5.41%   0.00%  2.94%  
 18    0   3   7.14%   9.52%  4.55%  
 19    0   2  14.29%   0.00%  9.52%  
 20    0   0   0.00%  33.33%  0.00%  ,
          下单日期  去重订单数  前置拦截     拦截率  进件数  预授权通过率  机审通过件  客户取消  人审拒绝   人审拒绝率  \
 16 2025-09-04     68     6   8.82%   38  55.88%     38     0    20  52.63%   
 17 2025-09-05     64    10  15.62%   37  57.81%     37     0    21  56.76%   
 18 2025-09-06     64     3   4.69%   42  65.62%     42   

## 小蚂蚁租机

In [361]:
# 小蚂蚁租机
xmy = all_models.merchant_names(df_contain,'小蚂蚁租机', '下单日期')
xmy = xmy[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xmy)!=0:
    xmy = clean.continuous_dates(xmy)
xmy_new = xmy.reset_index(names=['下单日期'])


## 乙辉数码

In [362]:
# 乙辉数码
yhsm = all_models.merchant_names(df_contain,'乙辉数码', '下单日期')
yhsm = yhsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# yhsm = clean.continuous_dates(yhsm)
yhsm_new = yhsm.reset_index()
yhsm_new.dtypes


下单日期      datetime64[ns]
去重订单数              int64
前置拦截             float64
拦截率              float64
进件数              float64
预授权通过率           float64
机审通过件            float64
客户取消             float64
人审拒绝             float64
人审拒绝率            float64
待审核              float64
出库               float64
进件出库率            float64
取消率              float64
订单出库率            float64
dtype: object

## 兴鑫兴通讯

In [363]:
# 兴鑫兴通讯
xxx = all_models.merchant_names(df_contain,'兴鑫兴通讯', '下单日期')
xxx = xxx[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xxx_new = xxx.reset_index()
# xxx_new

## 呱子笔记本电脑

In [364]:
# 呱子笔记本电脑
# gz = all_models.merchant_names(df_contain,'呱子笔记本电脑', '下单日期')
# gz = gz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# gz = clean.continuous_dates(gz)
# gz_new = gz.reset_index(names=['下单日期'])
# gz_new.set_index('下单日期')

## 南京聚格网络科技

In [421]:
jgwl = all_models.merchant_names(df_contain,'南京聚格网络科技', '下单日期')
jgwl = jgwl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
try:
    jgwl = clean.continuous_dates(jgwl)
except:
    print("warn: 没有数据")
jgwl_new = jgwl.reset_index(names=['下单日期'])
# jgwl#_new#.set_index('下单日期')

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
2025-08-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-08-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-09-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2025-09-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [366]:
# df_contain['merchant_name'].unique()


## 人人享租

In [367]:
# rrxz = all_models.merchant_names(df_contain,'人人享租', '下单日期')
# rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # rrxz = clean.continuous_dates(rrxz)
# rrxz_new = rrxz.reset_index(names=['下单日期'])
# # rrxz_new

try:
    rrxz = all_models.merchant_names(df_contain, '人人享租', '下单日期')
    if rrxz.empty:
        print("warn: 没有数据")
        rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
    else:
        rrxz = rrxz[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        # rrxz = clean.continuous_dates(rrxz)
        rrxz_new = rrxz.reset_index(names=['下单日期'])
except:
    rrxz_new = pd.DataFrame(columns=['去重订单数','前置拦截','拦截率','进件数','预授权通过率','机审通过件','客户取消','人审拒绝','人审拒绝率','待审核','出库','进件出库率','取消率','订单出库率'])
rrxz_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,58,0,0.00%,58,100.00%,0,0,0,0.00%,0,1,1.72%,0.00%,1.72%
17,2025-09-05,61,0,0.00%,61,100.00%,6,0,6,9.84%,0,2,3.28%,0.00%,3.28%
18,2025-09-06,55,0,0.00%,55,100.00%,4,0,4,7.27%,0,0,0.00%,0.00%,0.00%
19,2025-09-07,66,0,0.00%,66,100.00%,3,0,3,4.55%,0,0,0.00%,0.00%,0.00%
20,2025-09-08,28,0,0.00%,28,100.00%,5,0,2,7.14%,3,0,0.00%,0.00%,0.00%


## 崇胜数码

In [368]:
# cssm = all_models.merchant_names(df_contain,'崇胜数码', '下单日期')
# cssm = cssm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# cssm_new = cssm.reset_index(names=['下单日期'])

## 喜卓灵租机

In [369]:

xzl = all_models.merchant_names(df_contain,'喜卓灵租机', '下单日期')
xzl = xzl[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
if len(xzl)!=0:
    xzl = clean.continuous_dates(xzl)
xzl_new = xzl.reset_index(names=['下单日期'])

# xzl_new

## 喜卓灵新租机

In [370]:
# xzl_news = all_models.merchant_names(df_contain,'喜卓灵新租机', '下单日期')
# xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# # xzl_new = clean.continuous_dates(xzl_new)
# xzl_news = xzl_news.reset_index(names=['下单日期'])
# xzl_news

try:
    xzl_news = all_models.merchant_names(df_contain, '喜卓灵新租机', '下单日期')
    if xzl_news.empty:
        print("warn: 没有数据")
        xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xzl_news = xzl_news[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        # xzl_new = clean.continuous_dates(xzl_new)
        xzl_news = xzl_news.reset_index(names=['下单日期'])
except:
    xzl_news = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xzl_news.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,169,0,0.00%,169,100.00%,5,0,1,0.59%,0,9,5.33%,0.00%,5.33%
17,2025-09-05,189,0,0.00%,189,100.00%,19,0,19,10.05%,0,2,1.06%,0.00%,1.06%
18,2025-09-06,174,0,0.00%,173,99.43%,18,0,17,9.83%,0,2,1.16%,0.00%,1.15%
19,2025-09-07,154,0,0.00%,154,100.00%,3,0,2,1.30%,1,2,1.30%,0.00%,1.30%
20,2025-09-08,81,0,0.00%,81,100.00%,7,0,1,1.23%,5,1,1.23%,0.00%,1.23%


## 星晟数码

In [371]:
# # 入驻时间:2025-07-21 
# xcsm = all_models.merchant_names(df_contain,'星晟数码', '下单日期')
# xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# xcsm = clean.continuous_dates(xcsm)
# xcsm_new = xcsm.reset_index(names=['下单日期'])
# # xcsm_new

try:
    xcsm = all_models.merchant_names(df_contain, '星晟数码', '下单日期')
    if xcsm.empty:
        print("warn: 没有数据")
        xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        xcsm = xcsm[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        xcsm = clean.continuous_dates(xcsm)
        xcsm_new = xcsm.reset_index(names=['下单日期'])
except:
    xcsm_new = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
xcsm_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,40.0,0.0,0.00%,13.0,32.50%,13.0,0.0,1.0,7.69%,0.0,0.0,0.00%,0.00%,0.00%
17,2025-09-05,8.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
18,2025-09-06,4.0,0.0,0.00%,1.0,25.00%,1.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
19,2025-09-07,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
20,2025-09-08,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


### 支付宝订单

In [372]:
# df_contain['order_type'].value_counts()
df_contain_zfb = df_contain[df_contain['order_type'] == 'ZFB_ORDER'] 
xssm_zfb = all_models.merchant_names(df_contain_zfb,'星晟数码', '下单日期')
xssm_zfb = xssm_zfb[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_zfb = clean.continuous_dates(xssm_zfb)
xcsm_zfb_new = xssm_zfb.reset_index(names=['下单日期'])
xcsm_zfb_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,26.0,0.0,0.00%,2.0,7.69%,2.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
17,2025-09-05,8.0,0.0,0.00%,0.0,0.00%,0.0,0.0,0.0,nan%,0.0,0.0,nan%,nan%,0.00%
18,2025-09-06,4.0,0.0,0.00%,1.0,25.00%,1.0,0.0,0.0,0.00%,0.0,0.0,0.00%,0.00%,0.00%
19,2025-09-07,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
20,2025-09-08,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


### 京东订单

In [373]:
# df_contain['order_type'].value_counts()
df_contain_jd = df_contain[df_contain['order_type'] == 'JD_ORDER'] 
xssm_jd = all_models.merchant_names(df_contain_jd,'星晟数码', '下单日期')
xssm_jd = xssm_jd[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
xssm_jd = clean.continuous_dates(xssm_jd)
xcsm_jd_new = xssm_jd.reset_index(names=['下单日期'])
xcsm_jd_new.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,14.0,0.0,0.00%,11.0,78.57%,11.0,0.0,1.0,9.09%,0.0,0.0,0.00%,0.00%,0.00%
17,2025-09-05,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
18,2025-09-06,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
19,2025-09-07,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0
20,2025-09-08,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0


## 蘑菇时间  

In [374]:
# # 入驻时间:2025-07-08
# mgsj = all_models.merchant_names(df_contain,'蘑菇时间', '下单日期')
# mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
# mgsj = clean.continuous_dates(mgsj)
# mgsj_New = mgsj.reset_index(names=['下单日期'])
# # mgsj_New

try:
    mgsj = all_models.merchant_names(df_contain, '蘑菇时间', '下单日期')
    if mgsj.empty:
        print("warn: 没有数据")
        mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
    else:
        mgsj = mgsj[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"]]
        mgsj = clean.continuous_dates(mgsj)
        mgsj_New = mgsj.reset_index(names=['下单日期'])
except:
    mgsj_New = pd.DataFrame(columns=["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审通过件","客户取消",'人审拒绝','人审拒绝率','待审核',"出库","进件出库率","取消率","订单出库率"])
mgsj_New.tail()

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审通过件,客户取消,人审拒绝,人审拒绝率,待审核,出库,进件出库率,取消率,订单出库率
16,2025-09-04,3,0,0.00%,2,66.67%,2,0,0,0.00%,0,2,100.00%,0.00%,66.67%
17,2025-09-05,3,0,0.00%,2,66.67%,2,0,0,0.00%,0,1,50.00%,0.00%,33.33%
18,2025-09-06,4,0,0.00%,2,50.00%,2,0,0,0.00%,0,0,0.00%,0.00%,0.00%
19,2025-09-07,2,0,0.00%,1,50.00%,1,0,0,0.00%,0,0,0.00%,0.00%,0.00%
20,2025-09-08,1,0,0.00%,1,100.00%,1,0,0,0.00%,0,1,100.00%,0.00%,100.00%


# 每日各渠道订单统计 导出

In [375]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-09-08'

In [376]:
path = r'\\digua\迪瓜租机\20.日报数据相关输出/'
# 每日各渠道去重订单统计结果表格导出
# df = df[df.归属渠道=='搜索渠道']
gsqd_qcdd = pd.crosstab(df["下单日期"],df["归属渠道"],margins=True)
lyqd_qcdd = pd.crosstab(df["下单日期"],df["来源渠道"],margins=True)
hdmc_qcdd = pd.crosstab(df["下单日期"],df["activity_name"],margins=True)
#每日各渠道进件统计结果表格导出
# df_j = df_j[df_j.归属渠道=='搜索渠道']
gsqd_jj = pd.crosstab(df_j["下单日期"],df_j["归属渠道"],margins=True)
lyqd_jj = pd.crosstab(df_j["下单日期"],df_j["来源渠道"],margins=True)
hdmc_jj = pd.crosstab(df_j["下单日期"],df_j["activity_name"],margins=True)
# 各归属渠道 出库 导出
# dfck = dfck[dfck.归属渠道=='搜索渠道']
gsqd_ck = pd.crosstab(dfck["下单日期"],dfck["归属渠道"],margins=True)
lyqd_ck = pd.crosstab(dfck["下单日期"],dfck["来源渠道"],margins=True)
hdmc_ck = pd.crosstab(dfck["下单日期"],dfck["activity_name"],margins=True)
# 商品ID
qd_list = ['搜索-租手机','搜索-租苹果手机','搜索下挂-商品','搜索直达banner','搜索下挂券','顶部搜索框的搜索结果页','搜索侠客行-核心词','搜索侠客行-首页','搜索邦道-销售线索','搜索邦道-小程序推广']
sp_jj = pd.pivot_table(df_j[df_j.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
sp_ck = pd.pivot_table(dfck[dfck.来源渠道.isin(qd_list)],values='order_id', index=["下单日期", '商品ID'], columns='来源渠道', aggfunc='count')
try:
    with pd.ExcelWriter(path + f'日报相关数据输出_{t_date}.xlsx', engine='xlsxwriter') as writer:
        # 去重订单
        gsqd_qcdd.to_excel(writer, sheet_name='归属渠道去重订单统计', index=True)  
        lyqd_qcdd.to_excel(writer, sheet_name='来源渠道去重订单统计', index=True)
        hdmc_qcdd.to_excel(writer, sheet_name='活动名称去重订单统计', index=True)
        # 进件
        gsqd_jj.to_excel(writer, sheet_name='归属渠道进件统计', index=True)
        lyqd_jj.to_excel(writer, sheet_name='来源渠道进件统计', index=True)
        hdmc_jj.to_excel(writer, sheet_name='活动名称进件统计', index=True)
        # 出库
        gsqd_ck.to_excel(writer, sheet_name='归属渠道出库统计', index=True)
        lyqd_ck.to_excel(writer, sheet_name='来源渠道出库统计', index=True)
        hdmc_ck.to_excel(writer, sheet_name='活动名称出库统计', index=True)
        # 商品ID
        sp_jj.to_excel(writer, sheet_name='来源渠道商品ID进件统计', index=True)
        sp_ck.to_excel(writer, sheet_name='来源渠道商品ID出库统计', index=True)
except PermissionError:
    print(PermissionError)

<class 'PermissionError'>


In [377]:
# sp_jj.tail()
df["下单日期"].unique()

<DatetimeArray>
['2025-08-19 00:00:00', '2025-08-20 00:00:00', '2025-08-21 00:00:00',
 '2025-08-22 00:00:00', '2025-08-23 00:00:00', '2025-08-24 00:00:00',
 '2025-08-25 00:00:00', '2025-08-26 00:00:00', '2025-08-27 00:00:00',
 '2025-08-28 00:00:00', '2025-08-29 00:00:00', '2025-08-30 00:00:00',
 '2025-08-31 00:00:00', '2025-09-01 00:00:00', '2025-09-02 00:00:00',
 '2025-09-03 00:00:00', '2025-09-04 00:00:00', '2025-09-05 00:00:00',
 '2025-09-06 00:00:00', '2025-09-07 00:00:00', '2025-09-08 00:00:00']
Length: 21, dtype: datetime64[ns]

In [378]:
# dfck[(dfck['来源渠道']=='搜索下挂-商品')&(dfck['退货拒签']==1)].shape

# 总体 "租完即送"占比

In [379]:
# 获取拒量的出库和非拒量的出库
dfck_fjl = dfck[~dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
dfck_jl = dfck[(dfck.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(dfck.tips.str.contains('联合拒量订单')==False)]
# 可能哪天会发现连续15天没有拒量出库的数据，可以将时间范围扩大21-30天临时解决先跑下日报

In [380]:
dfck_jl#[dfck_jl['下单日期']=='2025-09-03']

,日期,订单号,类型,型号,内存_x,颜色_x,平台套餐,租金,已付租金,期数,订单归属,出租方,备注,快递单号,供应商,成本,碎屏险,设备串码,create_date,create_time,order_id,order_number,all_money,status,user_id,status2,channel_type,sku_attributes,product_name,new_actual_money,product_id,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,dy_order_item_json,type,buy_service_product,service_status,order_method,activity_id,appid,max_overdue_days,update_time,reason,purchase_amount,违约金,order_type,union_rent_tag,app_type,已收租金,物流状态,rejected_ato_risk,id,classify_id,商品ID,phone_name,下单日期,月份,下单月份,年份,hour,拒绝理由,取消原因,电审拒绝原因,无法联系原因,商品类型,租赁方案,押金类型,优惠券使用否,是否进行预授权,颜色_y,内存_y,进件,来源渠道,退货拒签,机型内存,归属渠道,状态编码,status_r,status_re,time_ra,status_ra,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,下单时段,待审核,前置拦截,人审拒绝,客户取消,无法联系,是否进件,是否出库,进件前取消,出库前风控强拒,机审强拒,机审通过件,风控通过件,已退款,是否二手,是否拒量,抖音电脑


In [381]:
# #总体 "租完即送"占比（拒量）
# df_rent_jlck = pd.crosstab(dfck_jl["下单日期"],dfck_jl["租赁方案"],margins=True)
# # 填充不连续日期的数据

# df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
# df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
# df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
# df_rent_jlck = df_rent_jlck.set_index('下单日期')
# df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
# try:
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完即送_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# try:
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
# except:
#     df_rent_jlck.loc[:, '租完归还_拒量'] = 0
#     df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))


In [382]:
import pandas as pd
from datetime import datetime, timedelta

df_rent_jlck = pd.crosstab(dfck_jl["下单日期"], dfck_jl["租赁方案"], margins=True)

# 如果 df_rent_jlck 为空，则进行处理
if df_rent_jlck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_rent_jlck = pd.DataFrame(index=date_range, columns=['租完即送_拒量', '租完归还_拒量', 'All_拒量', '租完即送出库占比_拒量', '租完即归还库占比_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_rent_jlck.fillna(0, inplace=True)
    # df_rent_jlck['租完即送出库占比_拒量'] = 0
    # df_rent_jlck['租完即归还库占比_拒量'] = 0
    
    # 设置索引名称
    df_rent_jlck.index.name = '下单日期'
else:
    # 原有处理逻辑
    # df_rent_jlck_clean = df_rent_jlck.iloc[:-1, :]#.dropna(subset=['下单日期']) 
    df_rent_jlck = clean.continuous_dates(df_rent_jlck.iloc[:-1, :])
    df_rent_jlck = df_rent_jlck.reset_index(names=['下单日期'])
    df_rent_jlck['下单日期'] = pd.to_datetime(df_rent_jlck['下单日期'])
    df_rent_jlck = df_rent_jlck.set_index('下单日期')
    df_rent_jlck.rename(columns={'租完即送': '租完即送_拒量', '租完归还': '租完归还_拒量', 'All': 'All_拒量'}, inplace=True)
    
    try:
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完即送_拒量'] = 0
        df_rent_jlck.loc[:, '租完即送出库占比_拒量'] = (df_rent_jlck.租完即送_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    
    try:
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))
    except:
        df_rent_jlck.loc[:, '租完归还_拒量'] = 0
        df_rent_jlck.loc[:, '租完即归还库占比_拒量'] = (df_rent_jlck.租完归还_拒量/df_rent_jlck.All_拒量).map(lambda x: format(x, '.2%'))

In [383]:
df_rent_jlck

,租完即送_拒量,租完归还_拒量,All_拒量,租完即送出库占比_拒量,租完即归还库占比_拒量
下单日期,,,,,
2025-08-24,0,0,0,0,0
2025-08-25,0,0,0,0,0
2025-08-26,0,0,0,0,0
2025-08-27,0,0,0,0,0
2025-08-28,0,0,0,0,0
2025-08-29,0,0,0,0,0
2025-08-30,0,0,0,0,0
2025-08-31,0,0,0,0,0
2025-09-01,0,0,0,0,0


In [384]:
#总体 "租完即送"占比（排除拒量）
df_rent  = pd.crosstab(df_j["下单日期"],df_j["租赁方案"],margins=True)
df_rent=df_rent.rename(columns={"租完即送":"租完即送进件","租完归还":"租完归还进件","All":"合计进件",})
df_rent_ck=pd.crosstab(dfck_fjl["下单日期"],dfck_fjl["租赁方案"],margins=True)
df_rent_ck=df_rent_ck.rename(columns={"租完即送":"租完即送出库","租完归还":"租完归还出库","All":"合计出库",})
df_r=pd.merge(df_rent,df_rent_ck,left_index=True, right_index=True)
df_r["租完即送进件占比"]=df_r["租完即送进件"]/df_r["合计进件"]
df_r["租完即送进件占比"]=df_r["租完即送进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还进件占比"]=df_r["租完归还进件"]/df_r["合计进件"]
df_r["租完归还进件占比"]=df_r["租完归还进件占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送出库占比"]=df_r["租完即送出库"]/df_r["合计出库"]
df_r["租完即送出库占比"]=df_r["租完即送出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完归还出库占比"]=df_r["租完归还出库"]/df_r["合计出库"]
df_r["租完归还出库占比"]=df_r["租完归还出库占比"].apply(lambda x:format(x,".2%"))
df_r["租完即送转化率"]=df_r["租完即送出库"]/df_r["租完即送进件"]
df_r["租完即送转化率"]=df_r["租完即送转化率"].apply(lambda x:format(x,".2%"))
df_r["租完归还转化率"]=df_r["租完归还出库"]/df_r["租完归还进件"]
df_r["租完归还转化率"]=df_r["租完归还转化率"].apply(lambda x:format(x,".2%"))
df_r = df_r.iloc[:-1, :]
df_r = df_r.reset_index()
df_r.loc[:, '下单日期'] = pd.to_datetime(df_r['下单日期'])
df_r.set_index('下单日期', inplace=True)
# 拼接拒量出库
df_r1 = df_r.merge(df_rent_jlck, on='下单日期', how='left')

df_r1 = df_r1[["租完即送进件","租完归还进件","租完归还进件占比","租完即送进件占比","租完即送出库","租完归还出库", '租完即送_拒量', '租完归还_拒量', '租完即归还库占比_拒量', '租完即送出库占比_拒量', "租完归还出库占比","租完即送出库占比", '租完即送转化率','租完归还转化率']]#.iloc[:-1, :]
df_r1 = clean.continuous_dates(df_r1)
df_r_new = df_r1.reset_index(names=['下单日期'])


# 总体出库订单碎屏险购买数据

## 总体

In [385]:
def ck(dfck, name):
    # '服务订单状态：1、待支付；2、待确认；3、服务中；4、已失效；5、已取消；6、已退款',
    dfck[f'已取消'] = np.where(dfck["service_status"]==5,1,0)
    dfck[f'已退款'] = np.where(dfck["service_status"]==6,1,0)
    dfck[f'待支付'] = np.where(dfck["service_status"]==1,1,0)
    dfck[f'待确认'] = np.where(dfck["service_status"]==2,1,0)
    dfck[f'服务中'] = np.where(dfck["service_status"]==3,1,0)

    df_s3  =  dfck.groupby(["下单日期"]).agg({f'已取消':'sum',f'已退款':'sum', f'待支付':'sum', f'待确认':'sum',f'服务中':'sum'})
    
    df_s3[f"最终实际支付碎屏险出库_{name}"]=df_s3[f"待确认"]+df_s3[f"服务中"] +df_s3[f"待支付"]
    return df_s3

In [386]:
# 不包含京享租
dfck_fjl_fjd = dfck_fjl[dfck_fjl.归属渠道!='京东渠道']
dfck_jl_fjd = dfck_jl[dfck_jl.归属渠道!='京东渠道']
#总体出库订单碎屏险购买数据

df_bx_ck=pd.crosstab(dfck_fjl_fjd["下单日期"],dfck_fjl_fjd["buy_service_product"],margins=True)

if df_bx_ck.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck.index.name = '下单日期'
else:
    df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
    
#总体出库订单碎屏险购买数据——拒量
df_bx_ck_jl=pd.crosstab(dfck_jl_fjd["下单日期"],dfck_jl_fjd["buy_service_product"],margins=True)

if df_bx_ck_jl.empty:
    # 获取当前日期
    current_date = datetime.now().date()
    
    # 计算15天前的日期
    start_date = current_date - timedelta(days=15)  # 15天不包括今天，所以是15天前
    
    # 生成最近15天的所有日期
    date_range = pd.date_range(start=start_date, end=current_date, freq='D')
    
    # 创建一个新的 DataFrame，列名与原始 df_rent_jlck 一致
    df_bx_ck_jl = pd.DataFrame(index=date_range, columns=['不购买碎屏险出库', '购买碎屏险出库', 'All', '合计出库_拒量'])
    
    # 将所有数据填充为 0 或 '0%'
    df_bx_ck_jl.fillna(0, inplace=True)
    
    
    # 设置索引名称
    df_bx_ck_jl.index.name = '下单日期'
else:

    df_bx_ck_jl=df_bx_ck_jl.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库_拒量",})

# 非拒量
df_s3 = ck(dfck_fjl_fjd, 'All')
# 拒量
df_s3_jl = ck(dfck_jl_fjd, '拒量')

# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
df_s3merge=pd.merge(df_bx_ck_jl,df_s3_jl,left_index=True, right_index=True)
df_s2merge = df_s2merge.merge(df_s3merge[['合计出库_拒量', '最终实际支付碎屏险出库_拒量']], left_index=True, right_index=True, how='left')
df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库_All"]/df_s2merge["合计出库"]
df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
df_s2merge.loc[:, '实际支付碎屏险比例_拒量'] = (df_s2merge.最终实际支付碎屏险出库_拒量/df_s2merge.合计出库_拒量).map(lambda x: format(x, '.2%'))

df_s2merge = df_s2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库', '合计出库_拒量','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库_拒量','最终实际支付碎屏险出库_All', '实际支付碎屏险比例_拒量',  '实际支付碎屏险比例']]
df_s2merge = clean.continuous_dates(df_s2merge)
df_s2merge = df_s2merge.reset_index(names=['下单日期'])
# df_s2merge
df_s2merge.tail()

,下单日期,不购买碎屏险出库,购买碎屏险出库,合计出库,合计出库_拒量,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库_拒量,最终实际支付碎屏险出库_All,实际支付碎屏险比例_拒量,实际支付碎屏险比例
16,2025-09-04,1.0,101.0,102.0,0.0,0.0,0.0,0.0,21.0,80.0,0.0,101.0,nan%,99.02%
17,2025-09-05,0.0,106.0,106.0,0.0,0.0,1.0,0.0,59.0,46.0,0.0,105.0,nan%,99.06%
18,2025-09-06,0.0,79.0,79.0,0.0,0.0,0.0,0.0,68.0,11.0,0.0,79.0,nan%,100.00%
19,2025-09-07,1.0,65.0,66.0,0.0,1.0,0.0,0.0,64.0,0.0,0.0,64.0,nan%,96.97%
20,2025-09-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [387]:
# df_s2merge
# dfck[dfck["归属渠道"]=="芝麻租物"]

In [388]:
#芝麻租物出库订单碎屏险购买数据
dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["buy_service_product"],margins=True)
df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})

dfckzw['已取消'] = np.where(dfckzw["service_status"]==5,1,0)
dfckzw['已退款'] = np.where(dfckzw["service_status"]==6,1,0)
dfckzw['待支付'] = np.where(dfckzw["service_status"]==1,1,0)
dfckzw['待确认'] = np.where(dfckzw["service_status"]==2,1,0)
dfckzw['服务中'] = np.where(dfckzw["service_status"]==3,1,0)

df_zws3  =   dfckzw.groupby(["下单日期"]).agg({'已取消':'sum','已退款':'sum', '待支付':'sum', '待确认':'sum','服务中':'sum'})  
# =pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})

df_zws2merge = pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
df_zws2merge["最终实际支付碎屏险出库"] = df_zws2merge["待确认"]+df_zws2merge["服务中"]+df_zws2merge["待支付"]
# 如果发现合计出库为0，或者没有芝麻租物的订单，则可能是每日各渠道订单统计导出中设置了出库渠道仅有搜索渠道
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["最终实际支付碎屏险出库"]/df_zws2merge["合计出库"]
df_zws2merge["实际支付碎屏险比例"] = df_zws2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))


df_zws2merge = df_zws2merge[['不购买碎屏险出库','购买碎屏险出库','合计出库','已取消','已退款', '待支付', '待确认','服务中','最终实际支付碎屏险出库','实际支付碎屏险比例']]
df_zws2merge = clean.continuous_dates(df_zws2merge)
df_zws2merge_new = df_zws2merge.reset_index(names=['下单日期1'])
# df_zws2merge_new = df_zws2merge_new.rename(columns={'下单日期': '下单日期1'})
df_zws2merge_new.tail()

,下单日期1,不购买碎屏险出库,购买碎屏险出库,合计出库,已取消,已退款,待支付,待确认,服务中,最终实际支付碎屏险出库,实际支付碎屏险比例
16,2025-09-04,0.0,61.0,61.0,0.0,0.0,0.0,13.0,48.0,61.0,100.00%
17,2025-09-05,0.0,54.0,54.0,0.0,1.0,0.0,29.0,24.0,53.0,98.15%
18,2025-09-06,0.0,44.0,44.0,0.0,0.0,0.0,40.0,4.0,44.0,100.00%
19,2025-09-07,1.0,39.0,40.0,0.0,0.0,0.0,39.0,0.0,39.0,97.50%
20,2025-09-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 整合

In [389]:
df_s2merge_new = df_s2merge#.reset_index()
df_s2merge_new.loc[:, '碎屏险实际出库占比'] = (df_s2merge_new.最终实际支付碎屏险出库_All/df_s2merge_new.购买碎屏险出库).map(lambda x: format(x,".2%"))
df_s2merge_new.loc[:, '非芝麻实际支付碎屏险比例'] = ((df_s2merge_new.购买碎屏险出库-df_zws2merge_new.购买碎屏险出库)/(df_s2merge_new.合计出库-df_zws2merge_new.合计出库)).map(lambda x: format(x,".2%"))
df_s2merge_all_new = pd.concat([df_s2merge_new, df_zws2merge_new], axis=1)
# df_s2merge_all_new#.set_index('下单日期')

## 京享租

In [390]:
df_jd2_ck = df_jd2[df_jd2.是否出库==1]
df_jd2_sp=pd.crosstab(df_jd2_ck["下单日期"],df_jd2_ck["buy_service_product"],margins=True)
df_jd2_sp=df_jd2_sp.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
df_jdsp = ck(df_jd2_ck, 'All')
df_jd_merge=pd.merge(df_jd2_sp,df_jdsp,left_index=True, right_index=True)
df_jd_merge["实际支付碎屏险比例"]=(df_jd_merge["最终实际支付碎屏险出库_All"]/df_jd_merge["合计出库"]).map(lambda x: format(x, '.2%'))
df_jd_merge = df_jd_merge.reset_index()
df_jd_merge.loc[:, '下单日期'] = df_jd_merge.下单日期.astype('object')
df_jd_group_new = df_jd_group_new.merge(df_jd_merge[['下单日期', '合计出库', '最终实际支付碎屏险出库_All', '实际支付碎屏险比例']], on='下单日期', how='left')
df_jd_group_new.tail()

,下单日期,创建订单数,创建进件数,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库（按下单时间）,出库（按转化时间）,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率,合计出库,最终实际支付碎屏险出库_All,实际支付碎屏险比例
16,2025-09-04,1715,757,1303,542,41.60%,705,54.11%,598.0,84.82%,107.0,49.0,58.0,8.23%,9,9,0.0,0,40,46.0,5.67%,1.28%,6.95%,0.00%,1.28%,3.07%,40,37,92.50%
17,2025-09-05,1624,670,1211,499,41.21%,636,52.52%,552.0,86.79%,84.0,38.0,46.0,7.23%,6,11,0.0,2,27,26.0,4.25%,0.94%,5.97%,0.00%,1.73%,2.23%,27,27,100.00%
18,2025-09-06,1413,634,1093,441,40.35%,604,55.26%,525.0,86.92%,79.0,41.0,38.0,6.29%,3,2,0.0,9,24,25.0,3.97%,0.50%,6.79%,0.00%,0.33%,2.20%,24,23,95.83%
19,2025-09-07,1318,520,1001,449,44.86%,497,49.65%,421.0,84.71%,76.0,39.0,37.0,7.44%,3,5,0.0,8,21,23.0,4.23%,0.60%,7.85%,0.00%,1.01%,2.10%,21,21,100.00%
20,2025-09-08,698,317,557,238,42.73%,305,54.76%,257.0,84.26%,48.0,12.0,36.0,11.80%,21,0,0.0,10,5,0.0,1.64%,6.89%,3.93%,0.00%,0.00%,0.90%,5,5,100.00%


# 拒量（廖志彧）

## 何薪华需求

In [391]:
# df.columns

In [392]:
# # 拒量的订单2024年12月到2025年6月正常还款的（租赁中）用户没有逾期的订单，需要订单号，用户下单的手机号码，审核人，还款期数（已还）
# df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&df.status==4]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# # 


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]

## 拒量数据(转化数据、明细)

In [393]:
# # # 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# # 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)
# df_name['分配人'].unique()


## 出库单分配人

In [394]:
# # # 分配人名单：蔡斯静 林婉婷 胡春燕 何薪华 刘慧琪 钟桂洪
# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪', '王伟鸿']
# # 筛选出库单分配人
# df_name = df_name[df_name['分配人'].isin(name_list)]
# # df_name.分配人.unique()
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]

# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 内连接，获取出库单分配人
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='inner')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{t_date}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# # 

In [395]:
# df_jl_new.下单日期.unique()

## 首逾

In [396]:
# # 在没有其它变化的情况下该口径是OK的：2025.9.3
# now_day = datetime.now()

# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]

# name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '钟桂洪','王伟鸿']
# df_name = df_name[df_name['分配人'].isin(name_list)]
# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# # 首逾
# date1 = '2025-07'
# date2 = '2025-08'
# df_new = df_jl[df_jl.下单月份==date1]

# # df_new["出库"] = np.where(df_new["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)

# df_new = df_new[df_new.是否出库==1]

# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# # df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='inner')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# # df_jl_stages_news

In [397]:
# df_jl.tail()

## wxz总体拒量程序

In [398]:
# 设置时间段
# now_day = datetime.now()
# seven_ago = now_day - pd.Timedelta(days=8)
# one_ago = now_day - pd.Timedelta(days=1)
# # 获取特定时间段的拒量订单的转化数据和明细
# # df_jl = df[(df.下单日期>=seven_ago)&(df.下单日期<=one_ago)&(df.tips.str.contains(r'策略2412|命中自有模型回捞策略')==True)] 多半无用
# df_jl = df[df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True]
# df_jl = df_jl[~df_jl.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机"])]
# df_jl_new = df_jl#[~df_jl.tips.str.contains('策略241205命中(1)', regex=False)]
# df_jl_new['策略命中等级'] = df_jl_new['tips'].str.extract(r'(策略241205命中\(\d+\)?|策略241212命中\(\d+\)?|命中自有模型回捞策略|回捞策略250330命中?)')[0]
# # 进件数，出库数，出库率，风险等级
# df_jl_new_group = df_jl_new.groupby('下单日期').agg({'是否进件': 'sum', '是否出库': 'sum'})
# df_jl_new_group.rename(columns={'是否进件': '进件数', '是否出库': '出库'}, inplace=True)
# df_jl_new_group['进件出库率'] = (df_jl_new_group.出库/df_jl_new_group.进件数).map(lambda x: format(x, '.2%'))
# df_jl_new2 = df_jl_new[['下单日期', 'order_number', '策略命中等级']]
# df_jl_new2_group = df_jl_new2.groupby('策略命中等级').agg(数量=('order_number', 'count'))

# 获取当前是星期几
# Monday = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
# if Monday == 'Monday':
#     with pd.ExcelWriter(f'F:/需求/廖志彧需求/拒量数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#         df_jl_new_group.to_excel(writer, sheet_name='转化数据')
#         df_jl_new2_group.to_excel(writer, sheet_name='拒量数据明细')
#         df_jl_new['order_number'].to_excel(writer, sheet_name='拒量明细', index=False)


# df_name_new = df_name.sort_values('update_time', ascending=False).groupby('order_id').head(1)
# df_jl_name = df_jl_new[df_jl_new.是否出库==1].merge(df_name_new, on='order_id', how='left')
# df_jl_name_new = df_jl_name[['下单日期', 'order_id', 'order_number', '策略命中等级', '分配人']]
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/分配数据_{Today_zfb}.xlsx', engine='openpyxl') as writer:
#     df_jl_name_new.to_excel(writer, sheet_name='出库单分配人', index=False)
# df_jl_name_new

# # 首逾
# date1 = '2025-06'
# date2 = '2025-07'
# df_new = df_jl[df_jl.下单月份==date1]
# df_new = df_new[df_new.是否出库==1]
# df_stages_new = df_stages[df_stages['order_id'].notna()]
# df_stages_new['order_id'] = df_stages_new['order_id'].astype(int)
# df_stages_new.loc[:, '预计还款月份'] = df_stages_new.refund_date.dt.strftime('%Y-%m')
# df_new_merge = df_new.merge(df_stages_new, on='order_id', how='left')
# df_jl_stages_new = df_new_merge[(df_new_merge.预计还款月份==date2)&(df_new_merge.sort==2)]
# df_jl_stages_new = df_new_merge[(df_new_merge.sort==2)]
# df_jl_stages_new.loc[:, 'now_day'] = pd.to_datetime(now_day.strftime('%Y-%m-%d'))
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.refund_date<=df_jl_stages_new.now_day]
# df_jl_stages_new.loc[:, 'overdue_day'] = np.where(df_jl_stages_new.reality_refund_date.notna(), 
#                                                 np.where(df_jl_stages_new.reality_refund_date>df_jl_stages_new.refund_date, (df_jl_stages_new.reality_refund_date-df_jl_stages_new.refund_date), 0)
#                                                 , np.where(df_jl_stages_new.refund_date>df_jl_stages_new.now_day, 0, (df_jl_stages_new.now_day-df_jl_stages_new.refund_date)))
# df_jl_stages_new.loc[:, 'overdue_day'] = df_jl_stages_new['overdue_day'].apply(lambda x: x.days)
# # df_jl_stages_new = df_jl_stages_new[df_jl_stages_new.overdue_day>0]

# # df_jl_stages_news = df_jl_stages_new[df_jl_stages_new.overdue_day>0][['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_new[['order_id', 'order_number', '下单日期', 'refund_date', 'reality_refund_date',  'overdue_day', 'status2']].rename(columns={'order_id': '订单id', 'order_number': '订单号', 'refund_date': '应还日期', 'reality_refund_date': '实还日期', 'overdue_day': '逾期天数'})
# df_jl_stages_news = df_jl_stages_news.merge(df_name_new[['order_id', '分配人']], left_on='订单id', right_on='order_id', how='left')
# df_jl_stages_news.loc[:, '是否逾期'] = np.where((df_jl_stages_news.status2=='租赁中')&(df_jl_stages_news.实还日期.isna())&(df_jl_stages_news.逾期天数>0), 1, 0)
# df_jl_stages_news_g = df_jl_stages_news.groupby('分配人').agg({'order_id': 'count', '是否逾期': 'sum'}).rename(columns={'order_id': '出库'})
# df_jl_stages_news_g.loc[:, '逾期/出库'] = (df_jl_stages_news_g.是否逾期/df_jl_stages_news_g.出库).map(lambda x: format(x, '.2%'))
# with pd.ExcelWriter(f'F:/需求/廖志彧需求/首逾_{t_date}.xlsx', engine='xlsxwriter') as writer:
#     df_jl_stages_news.to_excel(writer, sheet_name='出库订单', index=False)
#     df_jl_stages_news_g.to_excel(writer, sheet_name='首逾订单')
# df_jl_stages_news

In [399]:
# df_jl_new[['下单日期', 'order_number', '策略命中等级', '是否出库']].to_excel(f'F:/需求/总需求/拒量回捞数据_{t_date}.xlsx', index=False)
# df_jl_stages_news_g

## 拒量日报

In [400]:
df_jl = df[(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True)&(df.tips.str.contains('联合拒量订单')==False)]
# 剔除回捞被拒漏到商家的数据9.1
reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
df_jl = df_jl[~df_jl['merchant_name'].isin(reject_merchants)]

df_jl_group = df_jl.groupby('下单日期').agg({'是否进件': 'sum', '人审拒绝': 'sum', '风控通过件': 'sum', '客户取消': 'sum', '无法联系': 'sum', '出库前风控强拒': 'sum', '待审核': 'sum', '是否出库': 'sum', '已退款': 'sum'}).rename(columns={'是否进件': '进件数', '是否出库': '出库'})
df_jl_group.loc[:, '客户取消'] = df_jl_group.客户取消+df_jl_group.已退款
df_jl_group.loc[:, '风控通过率'] = (df_jl_group.风控通过件/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '进件出库率'] = (df_jl_group.出库/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '取消比例'] = (df_jl_group.客户取消/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '电审拒绝率'] = (df_jl_group.人审拒绝/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '出库前强拒比例'] = (df_jl_group.出库前风控强拒/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group.loc[:, '无法联系占比'] = (df_jl_group.无法联系/df_jl_group.进件数).map(lambda x: format(x, '.2%'))
df_jl_group_new = df_jl_group[['进件数', '人审拒绝', '风控通过件', '风控通过率', '客户取消', '无法联系', '出库前风控强拒', '待审核', '出库', '进件出库率', '取消比例', '电审拒绝率', '出库前强拒比例', '无法联系占比']].reset_index()

df_jl_group_new.tail()

,下单日期,进件数,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,进件出库率,取消比例,电审拒绝率,出库前强拒比例,无法联系占比
16,2025-09-04,98,72,20,20.41%,20,0,0,0,0,0.00%,20.41%,73.47%,0.00%,0.00%
17,2025-09-05,69,44,16,23.19%,15,0,1,0,0,0.00%,21.74%,63.77%,1.45%,0.00%
18,2025-09-06,87,73,9,10.34%,8,0,0,1,0,0.00%,9.20%,83.91%,0.00%,0.00%
19,2025-09-07,97,52,37,38.14%,36,0,0,1,0,0.00%,37.11%,53.61%,0.00%,0.00%
20,2025-09-08,38,9,25,65.79%,25,0,0,0,0,0.00%,65.79%,23.68%,0.00%,0.00%


# 挽留

In [401]:
df_wl_merge = df_wl.merge(df[['order_number', '是否出库']], on='order_number', how='left')
df_wl_merge.loc[:, '下单日期'] = pd.to_datetime(df_wl_merge.下单日期)
df_wl_merge.loc[:, '挽留成功'] = np.where(df_wl_merge.task_status=='SUCCESS', 1, 0)
df_wl_g = df_wl_merge.groupby('下单日期').agg({'order_number': 'nunique', '挽留成功': 'sum', '是否出库': 'sum'}).rename(columns={'order_number': '挽留数', '是否出库': '出库'})
df_wl_g_merge = df_all[['去重订单数', '进件数', '客户取消']].merge(df_wl_g, on='下单日期', how='left')
df_wl_g_merge.loc[:, '进件出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.进件数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '订单出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.去重订单数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留成功率'] = (df_wl_g_merge.挽留成功/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge.loc[:, '挽留出库率'] = (df_wl_g_merge.出库/df_wl_g_merge.挽留数).map(lambda x: format(x, '.2%'))
df_wl_g_merge = clean.continuous_dates(df_wl_g_merge)
df_wl_g_merge_new = df_wl_g_merge.reset_index(names=['下单日期'])
df_wl_g_merge_new.tail()

,下单日期,去重订单数,进件数,客户取消,挽留数,挽留成功,出库,进件出库率,订单出库率,挽留成功率,挽留出库率
16,2025-09-04,5829,1669,24,4.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%
17,2025-09-05,5616,1566,22,6.0,4.0,1.0,0.06%,0.02%,66.67%,16.67%
18,2025-09-06,4788,1429,25,3.0,1.0,0.0,0.00%,0.00%,33.33%,0.00%
19,2025-09-07,4611,1300,26,2.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%
20,2025-09-08,2595,746,97,0.0,0.0,0.0,nan%,nan%,nan%,nan%


# 获取日报文件名称并修改

In [402]:
now = datetime.now()
now_day = pd.to_datetime(now.strftime("%Y-%m-%d"))
now_date = now.strftime('%m%d')
before_date = (now-timedelta(days=1)).strftime('%m%d')
before_date_two = (now-timedelta(days=2)).strftime('%m%d')
before_date_four = (now-timedelta(days=4)).strftime('%m%d')
monday = pd.to_datetime(now.strftime('%Y-%m-%d')).day_name()

In [403]:
# 指定目录路径
directory = 'F:/日报/日报'

# 查找所有 .xlsx 和 .xls 文件
excel_files = glob.glob(os.path.join(directory, '*.xlsx')) + glob.glob(os.path.join(directory, '*.xls'))

for old_file_path in excel_files:
    # 提取文件名（不带路径）
    old_filename = os.path.basename(old_file_path)
    old_date = old_filename.split('.')[0][-4:]
    new_filename = old_filename.replace(old_date, before_date)
    # 将新的文件名添加到路径
    new_file_path = os.path.join(directory, new_filename)

    # 确保新文件名不存在，以避免覆盖文件
    if not os.path.exists(new_file_path):
        os.rename(old_file_path, new_file_path)
        print(f"文件 {old_filename} 已重命名为: {new_filename}")
    else:
        print(f"跳过文件 {old_filename}，因为新文件名已存在。")

跳过文件 new商户订单统计表截至0907.xlsx，因为新文件名已存在。
跳过文件 new迪瓜租机订单日报截止0907.xlsx，因为新文件名已存在。
跳过文件 ~$new迪瓜租机订单日报截止0907.xlsx，因为新文件名已存在。


# 读取日报

## 获取日期及星期

In [404]:

# 中文星期映射 获取星期几

# 设置需要打开的文件和密码
# file_path1 = f'F:/日报/日报/迪瓜租机订单日报截止{before_date}.xlsx'
file_path1 = f'F:/日报/日报/new迪瓜租机订单日报截止{before_date}.xlsx'

# file_path2 = f'F:/日报/日报/商户订单统计表截至{before_date}.xlsx'
file_path2 = f'F:/日报/日报/new商户订单统计表截至{before_date}.xlsx'
password = '20240101'

## 日报数据

In [405]:
def Open_Excel2(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新
    :param df: 日报各个渠道的df
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param sheet_name: excel的sheet页名称
    :param col: 列的位置，如A列，B列等
    :param col_len: 每个模块之间的间隔宽度
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key=='日报' else 0
        for idx, df_lis in enumerate(df_list):
            sheet = wb.sheets[sheet_names[idx+num]]
            # print(sheet_names[idx])
            col = col_list[idx]
            key = key_list[idx]
            for idx1, df in enumerate(df_lis):
                # 如果df没有数据则不执行操作
                # print(len(df))
                if len(df)!=0:
                    # print(len(df))
                    # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                    last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                    # 改进代码（查找特定列中的最后一个非空单元格）
                    # last_row = sheet.range(f'{col}{sheet.cells.last_cell.row}').end('up').row
                    # print(last_row)
                    # 更改
                    last_date = sheet.range(f'{col}{last_row}').value if key==0 else sheet.range(f'{col}{last_row - 4}').value
                    while last_date is None and last_row > 1: # 更改{
                        last_row -= 1
                        last_date = sheet.range(f'{col}{last_row}').value # 更改}
                    # 获取当前列的长度
                    col_len = sheet.range(f'{col}{1}').end('down').row
                    print(f"工作表: {sheet.name},current_key: {key}, col: {col}, last_row: {last_row}, last_date: {last_date}!")
                    # 获取前一天的日期
                    before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                    if col_len == 1:
                        if before_date != last_date:
                            # print(f'before_date: {before_date},last_date:{last_date}')
                            diff = (before_date - last_date).days
                            if key==0:
                                # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()

                    # 重新获取最后一行
                    last_row = sheet.used_range.last_cell.row
                    # 将 DataFrame 写入固定的位置
                    # 对应Excel中的行和列，起始行数
                    start_row = last_row - len(df) - 1 if key==0 else last_row - len(df) - 3
                    # print(start_row)
                    # 打印关键变量，便于调试
                    print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {key}, start_row: {start_row}")
                    # 定位写入区域左上角单元格（行, 列）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=False).value = df
                    # 获取上一个df的宽度并相加
                    col_len += 1 + len(df.columns)
            # 新的sheet页，数据列长重新赋值
            col_len = 1
        
        # 保存并重新加密文件
        # wb.save(password=password)
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()


## 创建新的sheet页

In [406]:



# # 自定义列名转数字的函数（替代xlwings.utils.col_name_to_num）
# def col_name_to_num(col_name):
#     """
#     将Excel列名（如'A', 'B', 'AA'）转换为数字（1, 2, 27）
#     """
#     num = 0
#     for c in col_name:
#         num = num * 26 + (ord(c.upper()) - ord('A') + 1)
#     return num

# def write_with_index_and_header(df_list, file_path, password, new_sheet_name, 
#                                start_col='A', start_row=1):
#     '''
#     创建新sheet页，同时写入行索引和列索引（表头），兼容所有xlwings版本
#     :param df_list: 待写入的DataFrame列表
#     :param file_path: Excel文件路径
#     :param password: Excel密码
#     :param new_sheet_name: 新sheet页名称
#     :param start_col: 起始列（默认'A'）
#     :param start_row: 起始行（默认1）
#     '''
#     app = xw.App(visible=False, add_book=False)
#     try:
#         # 打开加密文件
#         wb = xw.Book(file_path, password=password)
        
#         # 删除同名sheet（若存在）
#         for sheet in wb.sheets:
#             if sheet.name == new_sheet_name:
#                 sheet.delete()
        
#         # 创建新sheet
#         new_sheet = wb.sheets.add(name=new_sheet_name)
        
#         # 使用自定义函数将列名（如'A'）转换为数字（如1）
#         current_col = col_name_to_num(start_col)  # 替换原xlwings.utils的方法
        
#         for df in df_list:
#             if len(df) == 0:
#                 continue  # 跳过空数据
            
#             # 确定写入区域左上角（行=start_row，列=current_col）
#             target_range = new_sheet.range((start_row, current_col))
            
#             # 写入数据（保留行索引和列索引）
#             target_range.options(
#                 index=False,    # 不保留行索引
#                 header=True,   # 保留列索引（表头）
#                 chunksize=1000 # 大数据量优化
#             ).value = df
            
#             # 计算下一个数据的起始列（当前数据总列数 + 2列间隔）
#             total_cols = 1 + df.shape[1]  # 1（行索引） + 数据列数
#             current_col += total_cols + 2  # 加2列间隔
        
#         wb.save(password=password)
#         print(f"已成功写入新sheet《{new_sheet_name}》，包含行索引和列索引（表头）")
    
#     except Exception as e:
#         print(f"错误：{str(e)}")
#     finally:
#         wb.close()
#         app.quit()
        
# # 日报数据 - 在新sheet页写入
# # write_with_index_and_header([df_qudao_group[:-1]], file_path1, password, "拒收")

# # # 商家数据 - 在新sheet页写入
# # write_to_new_sheet(sj_df_list[0], file_path2, password, "商家数据汇总")

## 加入了写入空表的逻辑

In [407]:
def Open_Excel(df_list, path, password, col_list, key_list, key):
    '''
    对日报数据进行自动更新（增强版，处理空工作表）
    :param df_list: 日报各个渠道的df列表
    :param path: 日报的excel文件路径
    :param password: excel文件的密码
    :param col_list: 列的位置，如A列，B列等
    :param key_list: 每个sheet页的处理方式标志
    :param key: 判断是否有进行合计
    :return:
    '''
    # 定义文件路径和密码
    file_path = path
    password = password
    # 使用 xlwings 打开加密的 Excel 文件
    app = xw.App(visible=False)
    # 获取今天是星期几
    week = pd.to_datetime(datetime.now().strftime('%Y-%m-%d')).day_name()
    # 初始化数据列长
    col_len = 1
    try:
        wb = xw.Book(file_path, password=password)
        # 获取工作表
        sheet_names = wb.sheet_names
        num = 1 if key == '日报' else 0
        
        for idx, df_lis in enumerate(df_list):
            # 确保索引不越界
            if idx + num >= len(sheet_names):
                print(f"警告: 工作表索引 {idx+num} 超出范围，跳过")
                continue
                
            sheet = wb.sheets[sheet_names[idx+num]]
            col = col_list[idx]
            current_key = key_list[idx]  # 避免与函数参数key冲突
            
            # 检查工作表是否为空
            is_empty_sheet = sheet.used_range.last_cell.row == 1 and sheet.used_range.last_cell.column == 1
            
            for df in df_lis:
                if len(df) == 0:
                    continue
                
                if is_empty_sheet:
                    # 空工作表处理逻辑：直接写入数据，保留原格式
                    print(f"提示: 工作表 '{sheet.name}' 为空，直接写入数据")
                    start_row = 1  # 从第一行开始写入
                    print(f"工作表: {sheet.name}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                    # 写入数据（保留索引和表头）
                    target_range = sheet.range((start_row, col_len))
                    # 一次性写入整个DataFrame数据
                    target_range.options(index=False, header=True).value = df
                    
                    # 更新列位置
                    col_len += 1 + len(df.columns)
                    continue
                # 非空工作表的原有逻辑
                # 获取该工作表的最后一行，并插入一行
                    # col_len:判断是否是第一列第一个需要插入的数据；
                    # key：值等于1用来判断是否是免人审数据，免审订单转化，押金类型这三个数据页在最后一行的前三行插入； 值不等于1则在最后一行插入并且起始行数最后减4
                    # 获取最后一行的日期数据进行判断
                
                # 获取最后一行的日期数据进行判断
                last_row = sheet.range(col + str(sheet.cells.last_cell.row)).end('up').row
                last_date = sheet.range(f'{col}{last_row}').value if current_key==0 else sheet.range(f'{col}{last_row - 4}').value
                # 更改
                while last_date is None and sheet.name in ['拒收', '星晟数码', '蘑菇时间']: # 更改{
                    last_row -= 1                    
                    last_date = sheet.range(f'{col}{last_row}').value # 更改}
                print(f"工作表: {sheet.name},current_key: {current_key}, col:{col},last_row: {last_row}, last_date: {last_date}!")
                # if last_date is None:
                #     # 日期为空：插入一行并使用前一天作为默认日期 姑且看看
                #     if current_key == 0:
                # 非空工作表的原有逻辑
                # 获取最后一行
                # last_cell = sheet.range(col + str(sheet.cells.last_cell.row)).end('up')
                # last_row = last_cell.row无用
                
                # # 获取最后一行的日期
                # last_date_value = last_cell.value
                
                # # 处理日期值为空或无效的情况
                # if last_date_value is None:
                #     print(f"警告: 工作表 '{sheet.name}' 的日期列 {col} 为空，将插入新行")
                #     last_date = None
                # else:
                #     try:
                #         # 尝试将Excel日期值转换为pandas Timestamp
                #         if isinstance(last_date_value, str):
                #             # 处理字符串格式的日期
                #             last_date = pd.to_datetime(last_date_value).normalize()
                #         elif isinstance(last_date_value, (datetime, pd.Timestamp)):
                #             # 处理datetime或Timestamp对象
                #             last_date = pd.Timestamp(last_date_value).normalize()
                #         else:
                #             # 处理Excel数值格式的日期（浮点数）
                #             last_date = xw.utils.xl_to_datetime(last_date_value).normalize()
                #     except Exception as e:
                #         print(f"警告: 无法解析工作表 '{sheet.name}' 的日期值 '{last_date_value}': {str(e)}")
                #         last_date = None ！！！！！
                
                # 获取前一天的日期
                before_date = pd.to_datetime((datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'))
                
                # 处理日期为空或无效的情况
                if col_len == 1:
                    if last_date is None:
                        # 日期为空：插入一行并使用前一天作为默认日期
                        if current_key == 0:
                            sheet.api.Rows(last_row).Insert()
                        
                        else:
                            sheet.api.Rows(last_row - 3).Insert()
                        # 写入默认日期
                        sheet.range(f"{col}{last_row}").value = before_date.strftime('%Y-%m-%d')
                        last_date = before_date  # 更新last_date避免后续错误
                    # 判断最后一行日期是不是前一天的日期，如不是则差几天就插入几行，如是则不插入新的行
                    elif before_date != last_date:
                        # 日期不匹配：计算差值并插入相应行数
                        diff = (before_date - last_date).days
                        if diff > 0:  # 确保差值为正数
                            if current_key == 0:
                                sheet.api.Rows(last_row).Insert() if diff == 1 else sheet.api.Rows(f'{last_row}:{last_row + diff - 1}').Insert()
                            else:
                                sheet.api.Rows(last_row - 3).Insert() if diff == 1 else sheet.api.Rows(f'{last_row - diff + 1}:{last_row}').Insert()
                
                # 重新获取最后一行
                last_row = sheet.used_range.last_cell.row
                
                # 计算起始行
                start_row = last_row - len(df) - 1 if current_key == 0 else last_row - len(df) - 3
                start_row = max(start_row, 3)  # 确保行号至少为3，避免0或负数
                # 打印关键变量，便于调试
                print(f"写入时，工作表: {sheet.name}, last_row: {last_row}, len(df): {len(df)}, current_key: {current_key}, start_row: {start_row}")
                # 写入数据（保留索引）
                target_range = sheet.range((start_row, col_len))
                target_range.options(index=False, header=False).value = df
                
                # 更新列位置# 获取上一个df的宽度并相加
                col_len += 1 + len(df.columns)
            
            # 新的sheet页重置列位置
            col_len = 1
        
        # 保存并重新加密文件
        print("已保存并重新加密文件")
        wb.save(password=password)
        # # 确保关闭工作簿和应用程序实例
        # wb.close()
        # app.quit()
    finally:
        # 确保关闭工作簿和应用程序实例
        wb.close()
        app.quit()

## 将日报数据写入excel

In [408]:
# 日报数据
df_list = [[df_all_new[:-1], df_ss_group_new[:-1],df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]],
            [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
            [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]], [df_qudao_group[:]]]# , [df_qudao_group[:-1]]
col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

Open_Excel(df_list, file_path1, password, col_list, key_list, key='日报')
del df_list
gc.collect()
print('内存回收成功')

# # 日报数据
# df_list = [[df_all_new[:-1], df_ss_group_new[:-1], df_dr_group_new[:-1], df_zm_group_new[:-1], df_zw_group_new[:-1], df_tc[:-1], df_dy_group_new[:-1], df_zfb_group1[:-1]], 
#             [df_ms_new3[:-1]], [df_ms_o_group_new[:-1]],[df_qmy_group_new[:-1], df_fmy_group_new[:-1]], [df_r_new[:-1]], [df_s2merge_all_new[:-1]], [df_jl_group_new[:-1]], 
#             [df_wl_g_merge_new[:-1]], [df_jd_group_new[:-1], df_jdsc_group_new[:-1]]]
# col_list = ['C', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A']
# key_list = [0, 1, 1, 1, 0, 0, 0, 0, 0]
# Open_Excel2(df_list, file_path1, password, col_list, key_list, key='日报')
# 商家数据
sj_df_list = [[cxyz_new[:-1]], [hnw_new[:-1]], [zzy_new[:-1]], [qzsm_new[:-1]], [hkhz_new[:-1], hkhz_dy_new[:-1]], [xmy_new[:-1]], [rrxz_new[:-1]], [jgwl_new[:-1]], [xzl_new[:-1]],
                [xzl_news[:-1]], [xcsm_new[:-1], xcsm_zfb_new[:-1], xcsm_jd_new[:-1]], [mgsj_New[:-1]]]
sj_col_list = ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
sj_key_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Open_Excel(sj_df_list, file_path2, password, sj_col_list, sj_key_list, key='商家')
del sj_df_list
gc.collect()
print('内存回收成功')


工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 00:00:00!
写入时，工作表: 2025年, last_row: 255, len(df): 20, current_key: 0, start_row: 234
工作表: 2025年,current_key: 0, col:C,last_row: 253, last_date: 2025-09-07 

In [409]:
# sheet_name = '2025年'
# # 自动写入日报数据 总体

# col_len = all_models.Open_Excel2(df_all_new[:-1], file_path1, password, sheet_name, 'C')
# # 搜索渠道
# col_len1 = all_models.Open_Excel2(df_ss_group_new[:-1], file_path1, password, sheet_name, 'C', col_len)
# # 单人会话
# df_dr_group_new = df_dr_group.reset_index()
# col_len2 = all_models.Open_Excel2(df_dr_group_new[:-1], file_path1, password, sheet_name, 'C', col_len1)
# # 芝麻租物
# col_len3 = all_models.Open_Excel2(df_zm_group_new[:-1], file_path1, password, sheet_name, 'C', col_len2)
# # 纯租物
# df_zw_group_new = df_zw_group.reset_index()
# col_len4 = all_models.Open_Excel2(df_zw_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len3)
# # 总体剔除直播
# df_tc_new = df_tc.reset_index()
# col_len5 = all_models.Open_Excel2(df_tc_new[:-1], file_path1, password, sheet_name, 'C',  col_len4)
# # 抖音渠道
# df_dy_group_new = df_dy_group.reset_index()
# col_len6 = all_models.Open_Excel2(df_dy_group_new[:-1], file_path1, password, sheet_name, 'C', col_len5)
# #  支付宝直播
# df_zfb_group_new = df_zfb_group1.reset_index()
# col_len7 = all_models.Open_Excel2(df_zfb_group_new[:-1], file_path1, password, sheet_name, 'C',  col_len6)
# ## 免人审数据
# all_models.Open_Excel2(df_ms_new3[:-1], file_path1, password, '免人审数据', 'B',  key=1)
# ## 免审订单转化统计
# all_models.Open_Excel2(df_ms_o_group_new[:-1], file_path1, password, '免审订单转化统计', 'B',  key=1)
# # 全免押
# yj_col_len = all_models.Open_Excel2(df_qmy_group_new[:-1], file_path1, password, '押金类型', 'A',  key='押金')
# # # 非免押
# all_models.Open_Excel2(df_fmy_group_new[:-1], file_path1, password, '押金类型', 'A', col_len=yj_col_len, key='押金')
# ## 租完即送占比
# all_models.Open_Excel2(df_r_new[:-1], file_path1, password, '租完即送占比', 'A', key='租完即送')
# ## 碎屏险数据
# all_models.Open_Excel2(df_s2merge_all_new[:-1], file_path1, password, '碎屏险数据', 'A', key='碎屏险')
# ## 拒量
# all_models.Open_Excel2(df_jl_group_new[:-1], file_path1, password, '拒量', 'A', key='拒量')
# ## 挽留
# all_models.Open_Excel2(df_wl_g_merge_new[:-1], file_path1, password, '挽留', 'A', key='挽留')
# ## 京享租
# col_len = all_models.Open_Excel2(df_jd_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租')
# # 京东商城
# all_models.Open_Excel2(df_jdsc_group_new[:-1], file_path1, password, '京享租', 'A', key='京享租', col_len=col_len)


## 商家数据

In [410]:
# def merchant(df, name):
#     df_max_date = df.下单日期.max()
#     if df_max_date<(now_day-timedelta(days=1)):
#         df.loc[len(df)] = np.nan
#         # return df
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#     else:
#         all_models.Open_Excel2(df=df, path=file_path2, password=password, sheet_name=name, col='A', key='商家')
#         # return df

In [411]:
# # 澄心优租
# merchant(cxyz_new[:-1], '澄心优租')
# # 北京海鸟窝科技有限公司
# merchant(hnw_new[:-1], '海鸟窝')
# # 租着用
# merchant(zzy_new[:-1], '租着用')
# # 趣智数码
# merchant(qzsm_new[:-1], '趣智数码')
# # 汇客好租
# merchant(hkhz_new[:-1], '汇客好租')
# # 小蚂蚁租机
# merchant(xmy_new[:-1], '小蚂蚁租机')
# # 乙辉数码
# # merchant(yhsm_new, '乙辉数码')
# # 兴鑫兴通讯
# # merchant(xxx_new, '兴鑫兴通讯')

# # 南京聚格网络科技
# merchant(jgwl_new[:-1], '南京聚格网络科技')
# # 呱子笔记本电脑
# # merchant(gz_new[:-1], '呱子笔记本电脑')
# # 人人享租
# merchant(rrxz_new[:-1], '人人享租')
# # 崇胜数码
# # merchant(cssm_new[:-1], '崇胜数码')
# # 喜卓灵租机
# merchant(xzl_new[:-1], '喜卓灵租机')